In [1]:
import vectorbt as vbt
from vectorbt.portfolio.enums import *

import numpy as np
import pandas as pd
from numba import njit
from datetime import datetime, timedelta

seed = 42

price = pd.Series([1., 2., 3., 4., 5.], index=pd.Index([
    datetime(2020, 1, 1),
    datetime(2020, 1, 2),
    datetime(2020, 1, 3),
    datetime(2020, 1, 4),
    datetime(2020, 1, 5)
]))
price_wide = price.vbt.tile(3, keys=['a', 'b', 'c'])
print(price.shape, price_wide.shape)

big_price = pd.DataFrame(np.random.uniform(size=(1000,)))
big_price.index = [datetime(2018, 1, 1) + timedelta(days=i) for i in range(1000)]
big_price_wide = big_price.vbt.tile(1000)
print(big_price.shape, big_price_wide.shape)

# Disable caching for performance testing
# NOTE: Expect waterfall of executions, since some attributes depend on other attributes 
# that have to be calculated again and again
vbt.settings.caching['enabled'] = False

(5,) (5, 3)
(1000, 1) (1000, 1000)


## from_signals

In [2]:
entries = pd.Series([True, True, True, False, False], index=price.index)
entries_wide = entries.vbt.tile(3, keys=['a', 'b', 'c'])
print(entries.shape, entries_wide.shape)

exits = pd.Series([False, False, True, True, True], index=price.index)
exits_wide = exits.vbt.tile(3, keys=['a', 'b', 'c'])
print(exits.shape, exits_wide.shape)

big_entries = pd.DataFrame.vbt.signals.empty((1000,), index=big_price.index)
big_entries.iloc[0::2] = True
big_entries_wide = big_entries.vbt.tile(1000)
print(big_entries.shape, big_entries_wide.shape)

big_exits = pd.DataFrame.vbt.signals.empty((1000,), index=big_price.index)
big_exits.iloc[1::2] = True
big_exits_wide = big_exits.vbt.tile(1000)
print(big_exits.shape, big_exits_wide.shape)

(5,) (5, 3)
(5,) (5, 3)
(1000,) (1000, 1000)
(1000,) (1000, 1000)


In [3]:
%timeit vbt.Portfolio.from_signals(\
    big_price, big_entries, big_exits,\
    size=1., init_cash=np.inf, accumulate=True)

1.56 ms ± 162 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
%timeit vbt.Portfolio.from_signals(\
    big_price_wide, big_entries_wide, big_exits_wide,\
    size=1., init_cash=np.inf, accumulate=True)

65.9 ms ± 3.33 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
%timeit vbt.Portfolio.from_signals(\
    big_price_wide, big_entries_wide, big_exits_wide,\
    size=1., init_cash=np.inf, accumulate=True, log=True)

126 ms ± 36 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
%timeit vbt.Portfolio.from_signals(\
    big_price_wide, big_entries_wide, big_exits_wide,\
    size=1., init_cash=np.inf, accumulate=True,\
    group_by=np.repeat(np.arange(500), 2))

53.6 ms ± 176 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
%timeit vbt.Portfolio.from_signals(\
    big_price_wide, big_entries_wide, big_exits_wide,\
    size=1., init_cash=np.inf, accumulate=True,\
    group_by=np.repeat(np.arange(500), 2), cash_sharing=True)

54.2 ms ± 121 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
# worst case
%timeit vbt.Portfolio.from_signals(\
    big_price_wide, big_entries_wide, big_exits_wide,\
    size=np.arange(1000)[::-1], init_cash=np.inf, accumulate=True,\
    group_by=np.repeat(np.arange(500), 2), cash_sharing=True, call_seq=CallSeqType.Auto)

56.4 ms ± 412 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
%timeit vbt.Portfolio.from_signals(\
    big_price_wide, big_entries_wide, big_exits_wide,\
    size=1., init_cash=np.inf, accumulate=True,\
    group_by=np.full(1000, 0))

53.9 ms ± 243 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
%timeit vbt.Portfolio.from_signals(\
    big_price_wide, big_entries_wide, big_exits_wide,\
    size=1., init_cash=np.inf, accumulate=True,\
    group_by=np.full(1000, 0), cash_sharing=True)

49.5 ms ± 82.6 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
# worst case
%timeit vbt.Portfolio.from_signals(\
    big_price_wide, big_entries_wide, big_exits_wide,\
    size=np.arange(1000)[::-1], init_cash=np.inf, accumulate=True,\
    group_by=np.full(1000, 0), cash_sharing=True, call_seq=CallSeqType.Auto)

197 ms ± 1.75 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
def test_per_direction(price=price, entries=entries, exits=exits, **kwargs):
    print('All')
    try:
        portfolio = vbt.Portfolio.from_signals(price, entries, exits, direction='all', **kwargs)
        print(portfolio.orders.records)
    except Exception as e:
        print(e)
    print('LongOnly')
    try:
        portfolio = vbt.Portfolio.from_signals(price, entries, exits, direction='longonly', **kwargs)
        print(portfolio.orders.records)
    except Exception as e:
        print(e)
    print('ShortOnly')
    try:
        portfolio = vbt.Portfolio.from_signals(price, entries, exits, direction='shortonly', **kwargs)
        print(portfolio.orders.records)
    except Exception as e:
        print(e)

In [6]:
test_per_direction()

All
   id  idx  col   size  price  fees  side
0   0    0    0  100.0    1.0   0.0     0
1   1    3    0  200.0    4.0   0.0     1
LongOnly
   id  idx  col   size  price  fees  side
0   0    0    0  100.0    1.0   0.0     0
1   1    3    0  100.0    4.0   0.0     1
ShortOnly
   id  idx  col   size  price  fees  side
0   0    0    0  100.0    1.0   0.0     1
1   1    3    0   50.0    4.0   0.0     0


In [7]:
test_per_direction(size=[[-1, 0, 1, np.inf]])

All
   id  idx  col   size  price  fees  side
0   0    0    0    1.0    1.0   0.0     0
1   1    3    0    2.0    4.0   0.0     1
2   2    0    2    1.0    1.0   0.0     0
3   3    3    2    2.0    4.0   0.0     1
4   4    0    3  100.0    1.0   0.0     0
5   5    3    3  200.0    4.0   0.0     1
LongOnly
   id  idx  col   size  price  fees  side
0   0    0    0    1.0    1.0   0.0     0
1   1    3    0    1.0    4.0   0.0     1
2   2    0    2    1.0    1.0   0.0     0
3   3    3    2    1.0    4.0   0.0     1
4   4    0    3  100.0    1.0   0.0     0
5   5    3    3  100.0    4.0   0.0     1
ShortOnly
   id  idx  col   size  price  fees  side
0   0    0    0    1.0    1.0   0.0     1
1   1    3    0    1.0    4.0   0.0     0
2   2    0    2    1.0    1.0   0.0     1
3   3    3    2    1.0    4.0   0.0     0
4   4    0    3  100.0    1.0   0.0     1
5   5    3    3   50.0    4.0   0.0     0


In [8]:
test_per_direction(size=0.5, size_type='percent')

All
SizeType.Percent does not support Direction.All
LongOnly
   id  idx  col  size  price  fees  side
0   0    0    0  50.0    1.0   0.0     0
1   1    3    0  50.0    4.0   0.0     1
ShortOnly
   id  idx  col  size  price  fees  side
0   0    0    0  50.0    1.0   0.0     1
1   1    3    0  37.5    4.0   0.0     0


In [9]:
test_per_direction(size=0.5, size_type='percent', accumulate=True)

All
   id  idx  col    size  price  fees  side
0   0    0    0  50.000    1.0   0.0     0
1   1    1    0  12.500    2.0   0.0     0
2   2    3    0  65.625    4.0   0.0     1
3   3    4    0  25.625    5.0   0.0     1
LongOnly
   id  idx  col    size  price  fees  side
0   0    0    0  50.000    1.0   0.0     0
1   1    1    0  12.500    2.0   0.0     0
2   2    3    0  31.250    4.0   0.0     1
3   3    4    0  15.625    5.0   0.0     1
ShortOnly
   id  idx  col   size  price  fees  side
0   0    0    0  50.00    1.0   0.0     1
1   1    3    0  18.75    4.0   0.0     0
2   2    4    0   7.50    5.0   0.0     0


In [10]:
test_per_direction(price=price * 1.01)

All
   id  idx  col        size  price  fees  side
0   0    0    0   99.009901   1.01   0.0     0
1   1    3    0  198.019802   4.04   0.0     1
LongOnly
   id  idx  col       size  price  fees  side
0   0    0    0  99.009901   1.01   0.0     0
1   1    3    0  99.009901   4.04   0.0     1
ShortOnly
   id  idx  col       size  price  fees  side
0   0    0    0  99.009901   1.01   0.0     1
1   1    3    0  49.504950   4.04   0.0     0


In [11]:
test_per_direction(size=1, fees=[[0., 0.1, 1.]])

All
   id  idx  col  size  price  fees  side
0   0    0    0   1.0    1.0   0.0     0
1   1    3    0   2.0    4.0   0.0     1
2   2    0    1   1.0    1.0   0.1     0
3   3    3    1   2.0    4.0   0.8     1
4   4    0    2   1.0    1.0   1.0     0
5   5    3    2   2.0    4.0   8.0     1
LongOnly
   id  idx  col  size  price  fees  side
0   0    0    0   1.0    1.0   0.0     0
1   1    3    0   1.0    4.0   0.0     1
2   2    0    1   1.0    1.0   0.1     0
3   3    3    1   1.0    4.0   0.4     1
4   4    0    2   1.0    1.0   1.0     0
5   5    3    2   1.0    4.0   4.0     1
ShortOnly
   id  idx  col  size  price  fees  side
0   0    0    0   1.0    1.0   0.0     1
1   1    3    0   1.0    4.0   0.0     0
2   2    0    1   1.0    1.0   0.1     1
3   3    3    1   1.0    4.0   0.4     0
4   4    0    2   1.0    1.0   1.0     1
5   5    3    2   1.0    4.0   4.0     0


In [12]:
test_per_direction(size=1, fixed_fees=[[0., 0.1, 1.]])

All
   id  idx  col  size  price  fees  side
0   0    0    0   1.0    1.0   0.0     0
1   1    3    0   2.0    4.0   0.0     1
2   2    0    1   1.0    1.0   0.1     0
3   3    3    1   2.0    4.0   0.1     1
4   4    0    2   1.0    1.0   1.0     0
5   5    3    2   2.0    4.0   1.0     1
LongOnly
   id  idx  col  size  price  fees  side
0   0    0    0   1.0    1.0   0.0     0
1   1    3    0   1.0    4.0   0.0     1
2   2    0    1   1.0    1.0   0.1     0
3   3    3    1   1.0    4.0   0.1     1
4   4    0    2   1.0    1.0   1.0     0
5   5    3    2   1.0    4.0   1.0     1
ShortOnly
   id  idx  col  size  price  fees  side
0   0    0    0   1.0    1.0   0.0     1
1   1    3    0   1.0    4.0   0.0     0
2   2    0    1   1.0    1.0   0.1     1
3   3    3    1   1.0    4.0   0.1     0
4   4    0    2   1.0    1.0   1.0     1
5   5    3    2   1.0    4.0   1.0     0


In [13]:
test_per_direction(size=1, slippage=[[0., 0.1, 1.]])

All
   id  idx  col  size  price  fees  side
0   0    0    0   1.0    1.0   0.0     0
1   1    3    0   2.0    4.0   0.0     1
2   2    0    1   1.0    1.1   0.0     0
3   3    3    1   2.0    3.6   0.0     1
4   4    0    2   1.0    2.0   0.0     0
5   5    3    2   2.0    0.0   0.0     1
LongOnly
   id  idx  col  size  price  fees  side
0   0    0    0   1.0    1.0   0.0     0
1   1    3    0   1.0    4.0   0.0     1
2   2    0    1   1.0    1.1   0.0     0
3   3    3    1   1.0    3.6   0.0     1
4   4    0    2   1.0    2.0   0.0     0
5   5    3    2   1.0    0.0   0.0     1
ShortOnly
   id  idx  col  size  price  fees  side
0   0    0    0   1.0    1.0   0.0     1
1   1    3    0   1.0    4.0   0.0     0
2   2    0    1   1.0    0.9   0.0     1
3   3    3    1   1.0    4.4   0.0     0
4   4    0    2   1.0    0.0   0.0     1
5   5    3    2   1.0    8.0   0.0     0


In [14]:
test_per_direction(size=1, min_size=[[0., 1., 2.]])

All
   id  idx  col  size  price  fees  side
0   0    0    0   1.0    1.0   0.0     0
1   1    3    0   2.0    4.0   0.0     1
2   2    0    1   1.0    1.0   0.0     0
3   3    3    1   2.0    4.0   0.0     1
LongOnly
   id  idx  col  size  price  fees  side
0   0    0    0   1.0    1.0   0.0     0
1   1    3    0   1.0    4.0   0.0     1
2   2    0    1   1.0    1.0   0.0     0
3   3    3    1   1.0    4.0   0.0     1
ShortOnly
   id  idx  col  size  price  fees  side
0   0    0    0   1.0    1.0   0.0     1
1   1    3    0   1.0    4.0   0.0     0
2   2    0    1   1.0    1.0   0.0     1
3   3    3    1   1.0    4.0   0.0     0


In [15]:
test_per_direction(size=1, max_size=[[0.5, 1., np.inf]])

All
   id  idx  col  size  price  fees  side
0   0    0    0   0.5    1.0   0.0     0
1   1    3    0   0.5    4.0   0.0     1
2   2    4    0   0.5    5.0   0.0     1
3   3    0    1   1.0    1.0   0.0     0
4   4    3    1   1.0    4.0   0.0     1
5   5    4    1   1.0    5.0   0.0     1
6   6    0    2   1.0    1.0   0.0     0
7   7    3    2   2.0    4.0   0.0     1
LongOnly
   id  idx  col  size  price  fees  side
0   0    0    0   0.5    1.0   0.0     0
1   1    3    0   0.5    4.0   0.0     1
2   2    0    1   1.0    1.0   0.0     0
3   3    3    1   1.0    4.0   0.0     1
4   4    0    2   1.0    1.0   0.0     0
5   5    3    2   1.0    4.0   0.0     1
ShortOnly
   id  idx  col  size  price  fees  side
0   0    0    0   0.5    1.0   0.0     1
1   1    3    0   0.5    4.0   0.0     0
2   2    0    1   1.0    1.0   0.0     1
3   3    3    1   1.0    4.0   0.0     0
4   4    0    2   1.0    1.0   0.0     1
5   5    3    2   1.0    4.0   0.0     0


In [16]:
test_per_direction(size=1., reject_prob=[[0., 0.5, 1.]], seed=42)

All
   id  idx  col  size  price  fees  side
0   0    0    0   1.0    1.0   0.0     0
1   1    3    0   2.0    4.0   0.0     1
2   2    1    1   1.0    2.0   0.0     0
3   3    3    1   2.0    4.0   0.0     1
LongOnly
   id  idx  col  size  price  fees  side
0   0    0    0   1.0    1.0   0.0     0
1   1    3    0   1.0    4.0   0.0     1
2   2    1    1   1.0    2.0   0.0     0
3   3    3    1   1.0    4.0   0.0     1
ShortOnly
   id  idx  col  size  price  fees  side
0   0    0    0   1.0    1.0   0.0     1
1   1    3    0   1.0    4.0   0.0     0
2   2    1    1   1.0    2.0   0.0     1
3   3    3    1   1.0    4.0   0.0     0


In [25]:
portfolio = vbt.Portfolio.from_signals(
    price, entries, exits, direction='all',
    close_first=[[False, True]]
)
print(portfolio.orders.records)

portfolio = vbt.Portfolio.from_signals(
    pd.Series(price.values[::-1], index=price.index),
    pd.Series(entries.values[::-1], index=price.index),
    pd.Series(exits.values[::-1], index=price.index), 
    direction='all',
    close_first=[[False, True]]
)
print(portfolio.orders.records)

   id  idx  col   size  price  fees  side
0   0    0    0  100.0    1.0   0.0     0
1   1    3    0  200.0    4.0   0.0     1
2   2    0    1  100.0    1.0   0.0     0
3   3    3    1  100.0    4.0   0.0     1
4   4    4    1   80.0    5.0   0.0     1
   id  idx  col   size  price  fees  side
0   0    0    0   20.0    5.0   0.0     1
1   1    3    0  100.0    2.0   0.0     0
2   2    0    1   20.0    5.0   0.0     1
3   3    3    1   20.0    2.0   0.0     0
4   4    4    1  160.0    1.0   0.0     0


In [26]:
test_per_direction(size=1000, allow_partial=[[True, False]])
test_per_direction(size=np.inf, allow_partial=[[True, False]])

All
   id  idx  col    size  price  fees  side
0   0    0    0   100.0    1.0   0.0     0
1   1    3    0  1100.0    4.0   0.0     1
2   2    3    1  1000.0    4.0   0.0     1
LongOnly
   id  idx  col   size  price  fees  side
0   0    0    0  100.0    1.0   0.0     0
1   1    3    0  100.0    4.0   0.0     1
ShortOnly
   id  idx  col    size  price  fees  side
0   0    0    0  1000.0    1.0   0.0     1
1   1    3    0   275.0    4.0   0.0     0
2   2    0    1  1000.0    1.0   0.0     1
All
   id  idx  col   size  price  fees  side
0   0    0    0  100.0    1.0   0.0     0
1   1    3    0  200.0    4.0   0.0     1
2   2    0    1  100.0    1.0   0.0     0
3   3    3    1  200.0    4.0   0.0     1
LongOnly
   id  idx  col   size  price  fees  side
0   0    0    0  100.0    1.0   0.0     0
1   1    3    0  100.0    4.0   0.0     1
2   2    0    1  100.0    1.0   0.0     0
3   3    3    1  100.0    4.0   0.0     1
ShortOnly
   id  idx  col   size  price  fees  side
0   0    0    0  100.0

In [27]:
test_per_direction(size=1000, allow_partial=True, raise_reject=True)
test_per_direction(size=1000, allow_partial=False, raise_reject=True)

All
   id  idx  col    size  price  fees  side
0   0    0    0   100.0    1.0   0.0     0
1   1    3    0  1100.0    4.0   0.0     1
LongOnly
   id  idx  col   size  price  fees  side
0   0    0    0  100.0    1.0   0.0     0
1   1    3    0  100.0    4.0   0.0     1
ShortOnly
Order rejected: Not enough cash to cover fees
All
Order rejected: Final size is less than requested
LongOnly
Order rejected: Final size is less than requested
ShortOnly
Order rejected: Final size is less than requested


In [28]:
test_per_direction(size=1, accumulate=True)

All
   id  idx  col  size  price  fees  side
0   0    0    0   1.0    1.0   0.0     0
1   1    1    0   1.0    2.0   0.0     0
2   2    3    0   1.0    4.0   0.0     1
3   3    4    0   1.0    5.0   0.0     1
LongOnly
   id  idx  col  size  price  fees  side
0   0    0    0   1.0    1.0   0.0     0
1   1    1    0   1.0    2.0   0.0     0
2   2    3    0   1.0    4.0   0.0     1
3   3    4    0   1.0    5.0   0.0     1
ShortOnly
   id  idx  col  size  price  fees  side
0   0    0    0   1.0    1.0   0.0     1
1   1    1    0   1.0    2.0   0.0     1
2   2    3    0   1.0    4.0   0.0     0
3   3    4    0   1.0    5.0   0.0     0


In [29]:
portfolio = vbt.Portfolio.from_signals(price, entries, exits, log=True, direction='all')
print(portfolio.orders.records)
print(portfolio.logs.records)

   id  idx  col   size  price  fees  side
0   0    0    0  100.0    1.0   0.0     0
1   1    3    0  200.0    4.0   0.0     1
   id  idx  col  group  cash_now  shares_now  val_price_now  value_now  size  \
0   0    0    0      0     100.0         0.0            1.0      100.0   inf   
1   1    3    0      0       0.0       100.0            4.0      400.0  -inf   

   size_type  ...   log  new_cash  new_shares  res_size  res_price  res_fees  \
0          0  ...  True       0.0       100.0     100.0        1.0       0.0   
1          0  ...  True     800.0      -100.0     200.0        4.0       0.0   

   res_side  res_status  res_status_info  order_id  
0         0           0               -1         0  
1         1           0               -1         1  

[2 rows x 30 columns]


In [17]:
test_per_direction(price=price.iloc[:3], entries=pd.DataFrame([
    [True, True, True, True, True],
    [True, True, True, True, False],
    [True, True, True, True, True]
]), exits=pd.DataFrame([
    [True, True, True, True, True],
    [False, False, False, False, True],
    [True, True, True, True, True]
]), size=1., conflict_mode=[[
    'ignore', 
    'entry', 
    'exit', 
    'opposite',
    'opposite'
]])

All
   id  idx  col  size  price  fees  side
0   0    1    0   1.0    2.0   0.0     0
1   1    0    1   1.0    1.0   0.0     0
2   2    0    2   1.0    1.0   0.0     1
3   3    1    2   2.0    2.0   0.0     0
4   4    2    2   2.0    3.0   0.0     1
5   5    1    3   1.0    2.0   0.0     0
6   6    2    3   2.0    3.0   0.0     1
7   7    1    4   1.0    2.0   0.0     1
8   8    2    4   2.0    3.0   0.0     0
LongOnly
   id  idx  col  size  price  fees  side
0   0    1    0   1.0    2.0   0.0     0
1   1    0    1   1.0    1.0   0.0     0
2   2    1    2   1.0    2.0   0.0     0
3   3    2    2   1.0    3.0   0.0     1
4   4    1    3   1.0    2.0   0.0     0
5   5    2    3   1.0    3.0   0.0     1
ShortOnly
   id  idx  col  size  price  fees  side
0   0    1    0   1.0    2.0   0.0     1
1   1    0    1   1.0    1.0   0.0     1
2   2    1    2   1.0    2.0   0.0     1
3   3    2    2   1.0    3.0   0.0     0
4   4    1    3   1.0    2.0   0.0     1
5   5    2    3   1.0    3.0   0.0

In [18]:
test_per_direction(price=price_wide, size=1., init_cash=[0., 1., 100.])
test_per_direction(init_cash=np.inf)

All
   id  idx  col  size  price  fees  side
0   0    3    0   1.0    4.0   0.0     1
1   1    0    1   1.0    1.0   0.0     0
2   2    3    1   2.0    4.0   0.0     1
3   3    0    2   1.0    1.0   0.0     0
4   4    3    2   2.0    4.0   0.0     1
LongOnly
   id  idx  col  size  price  fees  side
0   0    0    1   1.0    1.0   0.0     0
1   1    3    1   1.0    4.0   0.0     1
2   2    0    2   1.0    1.0   0.0     0
3   3    3    2   1.0    4.0   0.0     1
ShortOnly
   id  idx  col  size  price  fees  side
0   0    0    0  1.00    1.0   0.0     1
1   1    3    0  0.25    4.0   0.0     0
2   2    0    1  1.00    1.0   0.0     1
3   3    3    1  0.50    4.0   0.0     0
4   4    0    2  1.00    1.0   0.0     1
5   5    3    2  1.00    4.0   0.0     0
All
Attempt to go in long direction indefinitely. Set max_size or finite init_cash.
LongOnly
Attempt to go in long direction indefinitely. Set max_size or finite init_cash.
ShortOnly
Attempt to go in short direction indefinitely. Set max_s

In [32]:
portfolio = vbt.Portfolio.from_signals(
    price_wide, entries, exits, 
    group_by=np.array([0, 0, 1]), direction='all')

print(portfolio.orders.records)

   id  idx  col   size  price  fees  side
0   0    0    0  100.0    1.0   0.0     0
1   1    3    0  200.0    4.0   0.0     1
2   2    0    1  100.0    1.0   0.0     0
3   3    3    1  200.0    4.0   0.0     1
4   4    0    2  100.0    1.0   0.0     0
5   5    3    2  200.0    4.0   0.0     1


In [33]:
portfolio = vbt.Portfolio.from_signals(
    price_wide, entries, exits, 
    group_by=np.array([0, 0, 1]), cash_sharing=True, direction='all')

print(portfolio.orders.records)
print(portfolio.call_seq)
try:
    print(portfolio.regroup(group_by=False))
except Exception as e:
    print(e)

   id  idx  col   size  price  fees  side
0   0    0    0  100.0    1.0   0.0     0
1   1    3    0  200.0    4.0   0.0     1
2   2    3    1  200.0    4.0   0.0     1
3   3    0    2  100.0    1.0   0.0     0
4   4    3    2  200.0    4.0   0.0     1
            a  b  c
2020-01-01  0  1  0
2020-01-02  0  1  0
2020-01-03  0  1  0
2020-01-04  0  1  0
2020-01-05  0  1  0
Cannot modify grouping globally when cash_sharing=True


In [34]:
portfolio = vbt.Portfolio.from_signals(
    price_wide, entries, exits, 
    group_by=np.array([0, 0, 1]), cash_sharing=True, direction='all', call_seq='reversed')

print(portfolio.orders.records)
print(portfolio.call_seq)

   id  idx  col   size  price  fees  side
0   0    0    1  100.0    1.0   0.0     0
1   1    3    1  200.0    4.0   0.0     1
2   2    3    0  200.0    4.0   0.0     1
3   3    0    2  100.0    1.0   0.0     0
4   4    3    2  200.0    4.0   0.0     1
            a  b  c
2020-01-01  1  0  0
2020-01-02  1  0  0
2020-01-03  1  0  0
2020-01-04  1  0  0
2020-01-05  1  0  0


In [35]:
portfolio = vbt.Portfolio.from_signals(
    price_wide, entries, exits, 
    group_by=np.array([0, 0, 1]), cash_sharing=True, direction='all', call_seq='random')

print(portfolio.orders.records)
print(portfolio.call_seq)

portfolio = vbt.Portfolio.from_signals(
    price_wide, entries, exits, 
    group_by=np.array([0, 0, 1]), cash_sharing=True, direction='all', call_seq='random', seed=42)

print(portfolio.orders.records)
print(portfolio.call_seq)

   id  idx  col   size  price  fees  side
0   0    0    1  100.0    1.0   0.0     0
1   1    3    1  200.0    4.0   0.0     1
2   2    3    0  200.0    4.0   0.0     1
3   3    0    2  100.0    1.0   0.0     0
4   4    3    2  200.0    4.0   0.0     1
            a  b  c
2020-01-01  1  0  0
2020-01-02  1  0  0
2020-01-03  1  0  0
2020-01-04  1  0  0
2020-01-05  0  1  0
   id  idx  col   size  price  fees  side
0   0    0    1  100.0    1.0   0.0     0
1   1    3    1  200.0    4.0   0.0     1
2   2    3    0  200.0    4.0   0.0     1
3   3    0    2  100.0    1.0   0.0     0
4   4    3    2  200.0    4.0   0.0     1
            a  b  c
2020-01-01  1  0  0
2020-01-02  0  1  0
2020-01-03  1  0  0
2020-01-04  1  0  0
2020-01-05  1  0  0


In [36]:
for direction in ('All', 'LongOnly', 'ShortOnly'):
    print(direction)
    portfolio = vbt.Portfolio.from_signals(1., pd.DataFrame([
        [False, False, True],
        [False, True, False],
        [True, False, False],
        [False, False, True],
        [False, True, False],
    ]), pd.DataFrame([
        [False, False, False],
        [False, False, True],
        [False, True, False],
        [True, False, False],
        [False, False, True],
    ]), group_by=np.array([0, 0, 0]), 
        cash_sharing=True, call_seq='auto', direction=direction)

    print(portfolio.orders.records)
    print(portfolio.call_seq)
    print(portfolio.shares())
    print(portfolio.value())
    # notice how shorting with np.inf only accounts for current cash, not for other assets!

All
   id  idx  col    size  price  fees  side
0   0    0    2   100.0    1.0   0.0     0
1   1    1    2   200.0    1.0   0.0     1
2   2    1    1   200.0    1.0   0.0     0
3   3    2    1   400.0    1.0   0.0     1
4   4    2    0   400.0    1.0   0.0     0
5   5    3    0   800.0    1.0   0.0     1
6   6    3    2   800.0    1.0   0.0     0
7   7    4    2  1400.0    1.0   0.0     1
8   8    4    1  1400.0    1.0   0.0     0
   0  1  2
0  0  1  2
1  2  0  1
2  1  2  0
3  0  1  2
4  2  0  1
       0       1      2
0    0.0     0.0  100.0
1    0.0   200.0 -100.0
2  400.0  -200.0 -100.0
3 -400.0  -200.0  700.0
4 -400.0  1200.0 -700.0
0    100.0
1    100.0
2    100.0
3    100.0
4    100.0
dtype: float64
LongOnly
   id  idx  col   size  price  fees  side
0   0    0    2  100.0    1.0   0.0     0
1   1    1    2  100.0    1.0   0.0     1
2   2    1    1  100.0    1.0   0.0     0
3   3    2    1  100.0    1.0   0.0     1
4   4    2    0  100.0    1.0   0.0     0
5   5    3    0  100.0   

## from_orders

In [19]:
order_size = pd.Series([np.inf, -np.inf, np.nan, np.inf, -np.inf], index=price.index)
order_size_wide = order_size.vbt.tile(3, keys=['a', 'b', 'c'])

big_order_size = pd.DataFrame.vbt.empty((1000,), 1, dtype=np.float_)
big_order_size.iloc[1::2] = -1
big_order_size_wide = big_order_size.vbt.tile(1000)

In [20]:
%timeit vbt.Portfolio.from_orders(big_price, big_order_size, init_cash=np.inf)

1.5 ms ± 177 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
%timeit vbt.Portfolio.from_orders(big_price_wide, big_order_size_wide, init_cash=np.inf)

49.2 ms ± 1.68 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
%timeit vbt.Portfolio.from_orders(\
    big_price_wide, big_order_size_wide, init_cash=np.inf, \
    group_by=np.repeat(np.arange(500), 2))

46.2 ms ± 2.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [41]:
%timeit vbt.Portfolio.from_orders(\
    big_price_wide, big_order_size_wide, init_cash=np.inf, \
    group_by=np.repeat(np.arange(500), 2), cash_sharing=True)

43.5 ms ± 1.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [42]:
%timeit vbt.Portfolio.from_orders(\
    big_price_wide, big_order_size_wide, init_cash=np.inf, \
    group_by=np.repeat(np.arange(500), 2), cash_sharing=True, call_seq=CallSeqType.Auto)

50 ms ± 725 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [43]:
%timeit vbt.Portfolio.from_orders(\
    big_price_wide, big_order_size_wide, init_cash=np.inf, \
    group_by=np.full(1000, 0))

43.4 ms ± 769 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [44]:
%timeit vbt.Portfolio.from_orders(\
    big_price_wide, big_order_size_wide, init_cash=np.inf, \
    group_by=np.full(1000, 0), cash_sharing=True)

39.7 ms ± 122 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [45]:
%timeit vbt.Portfolio.from_orders(\
    big_price_wide, big_order_size_wide, init_cash=np.inf, \
    group_by=np.full(1000, 0), cash_sharing=True, call_seq=CallSeqType.Auto)

45.7 ms ± 1.03 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [23]:
def test_per_direction(price=price, size=order_size, **kwargs):
    print('All')
    try:
        portfolio = vbt.Portfolio.from_orders(price, size, direction='all', **kwargs)
        print(portfolio.orders.records)
    except Exception as e:
        print(e)
    print('LongOnly')
    try:
        portfolio = vbt.Portfolio.from_orders(price, size, direction='longonly', **kwargs)
        print(portfolio.orders.records)
    except Exception as e:
        print(e)
    print('ShortOnly')
    try:
        portfolio = vbt.Portfolio.from_orders(price, size, direction='shortonly', **kwargs)
        print(portfolio.orders.records)
    except Exception as e:
        print(e)

In [24]:
test_per_direction()

All
   id  idx  col   size  price  fees  side
0   0    0    0  100.0    1.0   0.0     0
1   1    1    0  200.0    2.0   0.0     1
2   2    3    0  100.0    4.0   0.0     0
LongOnly
   id  idx  col   size  price  fees  side
0   0    0    0  100.0    1.0   0.0     0
1   1    1    0  100.0    2.0   0.0     1
2   2    3    0   50.0    4.0   0.0     0
3   3    4    0   50.0    5.0   0.0     1
ShortOnly
   id  idx  col   size  price  fees  side
0   0    0    0  100.0    1.0   0.0     1
1   1    1    0  100.0    2.0   0.0     0


In [25]:
test_per_direction(price=price_wide)

All
   id  idx  col   size  price  fees  side
0   0    0    0  100.0    1.0   0.0     0
1   1    1    0  200.0    2.0   0.0     1
2   2    3    0  100.0    4.0   0.0     0
3   3    0    1  100.0    1.0   0.0     0
4   4    1    1  200.0    2.0   0.0     1
5   5    3    1  100.0    4.0   0.0     0
6   6    0    2  100.0    1.0   0.0     0
7   7    1    2  200.0    2.0   0.0     1
8   8    3    2  100.0    4.0   0.0     0
LongOnly
    id  idx  col   size  price  fees  side
0    0    0    0  100.0    1.0   0.0     0
1    1    1    0  100.0    2.0   0.0     1
2    2    3    0   50.0    4.0   0.0     0
3    3    4    0   50.0    5.0   0.0     1
4    4    0    1  100.0    1.0   0.0     0
5    5    1    1  100.0    2.0   0.0     1
6    6    3    1   50.0    4.0   0.0     0
7    7    4    1   50.0    5.0   0.0     1
8    8    0    2  100.0    1.0   0.0     0
9    9    1    2  100.0    2.0   0.0     1
10  10    3    2   50.0    4.0   0.0     0
11  11    4    2   50.0    5.0   0.0     1
ShortOnl

In [26]:
test_per_direction(size=[[np.inf, -np.inf]])

All
   id  idx  col   size  price  fees  side
0   0    0    0  100.0    1.0   0.0     0
1   1    0    1  100.0    1.0   0.0     1
LongOnly
   id  idx  col   size  price  fees  side
0   0    0    0  100.0    1.0   0.0     0
ShortOnly
   id  idx  col   size  price  fees  side
0   0    0    0  100.0    1.0   0.0     1


In [27]:
order_size_one = pd.Series([1, -1, np.nan, 1, -1], index=price.index)

In [28]:
test_per_direction(price=price * 1.01)

All
   id  idx  col        size  price  fees  side
0   0    0    0   99.009901   1.01   0.0     0
1   1    1    0  198.019802   2.02   0.0     1
2   2    3    0   99.009901   4.04   0.0     0
LongOnly
   id  idx  col       size  price  fees  side
0   0    0    0  99.009901   1.01   0.0     0
1   1    1    0  99.009901   2.02   0.0     1
2   2    3    0  49.504950   4.04   0.0     0
3   3    4    0  49.504950   5.05   0.0     1
ShortOnly
   id  idx  col       size  price  fees  side
0   0    0    0  99.009901   1.01   0.0     1
1   1    1    0  99.009901   2.02   0.0     0


In [29]:
test_per_direction(size=order_size_one, fees=[[0., 0.1, 1.]])

All
    id  idx  col  size  price  fees  side
0    0    0    0   1.0    1.0   0.0     0
1    1    1    0   1.0    2.0   0.0     1
2    2    3    0   1.0    4.0   0.0     0
3    3    4    0   1.0    5.0   0.0     1
4    4    0    1   1.0    1.0   0.1     0
5    5    1    1   1.0    2.0   0.2     1
6    6    3    1   1.0    4.0   0.4     0
7    7    4    1   1.0    5.0   0.5     1
8    8    0    2   1.0    1.0   1.0     0
9    9    1    2   1.0    2.0   2.0     1
10  10    3    2   1.0    4.0   4.0     0
11  11    4    2   1.0    5.0   5.0     1
LongOnly
    id  idx  col  size  price  fees  side
0    0    0    0   1.0    1.0   0.0     0
1    1    1    0   1.0    2.0   0.0     1
2    2    3    0   1.0    4.0   0.0     0
3    3    4    0   1.0    5.0   0.0     1
4    4    0    1   1.0    1.0   0.1     0
5    5    1    1   1.0    2.0   0.2     1
6    6    3    1   1.0    4.0   0.4     0
7    7    4    1   1.0    5.0   0.5     1
8    8    0    2   1.0    1.0   1.0     0
9    9    1    2   1.

In [30]:
test_per_direction(size=order_size_one, fixed_fees=[[0., 0.1, 1.]])

All
    id  idx  col  size  price  fees  side
0    0    0    0   1.0    1.0   0.0     0
1    1    1    0   1.0    2.0   0.0     1
2    2    3    0   1.0    4.0   0.0     0
3    3    4    0   1.0    5.0   0.0     1
4    4    0    1   1.0    1.0   0.1     0
5    5    1    1   1.0    2.0   0.1     1
6    6    3    1   1.0    4.0   0.1     0
7    7    4    1   1.0    5.0   0.1     1
8    8    0    2   1.0    1.0   1.0     0
9    9    1    2   1.0    2.0   1.0     1
10  10    3    2   1.0    4.0   1.0     0
11  11    4    2   1.0    5.0   1.0     1
LongOnly
    id  idx  col  size  price  fees  side
0    0    0    0   1.0    1.0   0.0     0
1    1    1    0   1.0    2.0   0.0     1
2    2    3    0   1.0    4.0   0.0     0
3    3    4    0   1.0    5.0   0.0     1
4    4    0    1   1.0    1.0   0.1     0
5    5    1    1   1.0    2.0   0.1     1
6    6    3    1   1.0    4.0   0.1     0
7    7    4    1   1.0    5.0   0.1     1
8    8    0    2   1.0    1.0   1.0     0
9    9    1    2   1.

In [31]:
test_per_direction(size=order_size_one, slippage=[[0., 0.1, 1.]])

All
    id  idx  col  size  price  fees  side
0    0    0    0   1.0    1.0   0.0     0
1    1    1    0   1.0    2.0   0.0     1
2    2    3    0   1.0    4.0   0.0     0
3    3    4    0   1.0    5.0   0.0     1
4    4    0    1   1.0    1.1   0.0     0
5    5    1    1   1.0    1.8   0.0     1
6    6    3    1   1.0    4.4   0.0     0
7    7    4    1   1.0    4.5   0.0     1
8    8    0    2   1.0    2.0   0.0     0
9    9    1    2   1.0    0.0   0.0     1
10  10    3    2   1.0    8.0   0.0     0
11  11    4    2   1.0    0.0   0.0     1
LongOnly
    id  idx  col  size  price  fees  side
0    0    0    0   1.0    1.0   0.0     0
1    1    1    0   1.0    2.0   0.0     1
2    2    3    0   1.0    4.0   0.0     0
3    3    4    0   1.0    5.0   0.0     1
4    4    0    1   1.0    1.1   0.0     0
5    5    1    1   1.0    1.8   0.0     1
6    6    3    1   1.0    4.4   0.0     0
7    7    4    1   1.0    4.5   0.0     1
8    8    0    2   1.0    2.0   0.0     0
9    9    1    2   1.

In [32]:
test_per_direction(size=order_size_one, min_size=[[0., 1., 2.]])

All
   id  idx  col  size  price  fees  side
0   0    0    0   1.0    1.0   0.0     0
1   1    1    0   1.0    2.0   0.0     1
2   2    3    0   1.0    4.0   0.0     0
3   3    4    0   1.0    5.0   0.0     1
4   4    0    1   1.0    1.0   0.0     0
5   5    1    1   1.0    2.0   0.0     1
6   6    3    1   1.0    4.0   0.0     0
7   7    4    1   1.0    5.0   0.0     1
LongOnly
   id  idx  col  size  price  fees  side
0   0    0    0   1.0    1.0   0.0     0
1   1    1    0   1.0    2.0   0.0     1
2   2    3    0   1.0    4.0   0.0     0
3   3    4    0   1.0    5.0   0.0     1
4   4    0    1   1.0    1.0   0.0     0
5   5    1    1   1.0    2.0   0.0     1
6   6    3    1   1.0    4.0   0.0     0
7   7    4    1   1.0    5.0   0.0     1
ShortOnly
   id  idx  col  size  price  fees  side
0   0    0    0   1.0    1.0   0.0     1
1   1    1    0   1.0    2.0   0.0     0
2   2    3    0   1.0    4.0   0.0     1
3   3    4    0   1.0    5.0   0.0     0
4   4    0    1   1.0    1.0   0.0

In [33]:
test_per_direction(size=order_size_one, max_size=[[0.5, 1., np.inf]])

All
    id  idx  col  size  price  fees  side
0    0    0    0   0.5    1.0   0.0     0
1    1    1    0   0.5    2.0   0.0     1
2    2    3    0   0.5    4.0   0.0     0
3    3    4    0   0.5    5.0   0.0     1
4    4    0    1   1.0    1.0   0.0     0
5    5    1    1   1.0    2.0   0.0     1
6    6    3    1   1.0    4.0   0.0     0
7    7    4    1   1.0    5.0   0.0     1
8    8    0    2   1.0    1.0   0.0     0
9    9    1    2   1.0    2.0   0.0     1
10  10    3    2   1.0    4.0   0.0     0
11  11    4    2   1.0    5.0   0.0     1
LongOnly
    id  idx  col  size  price  fees  side
0    0    0    0   0.5    1.0   0.0     0
1    1    1    0   0.5    2.0   0.0     1
2    2    3    0   0.5    4.0   0.0     0
3    3    4    0   0.5    5.0   0.0     1
4    4    0    1   1.0    1.0   0.0     0
5    5    1    1   1.0    2.0   0.0     1
6    6    3    1   1.0    4.0   0.0     0
7    7    4    1   1.0    5.0   0.0     1
8    8    0    2   1.0    1.0   0.0     0
9    9    1    2   1.

In [34]:
test_per_direction(size=order_size_one, reject_prob=[[0., 0.5, 1.]], seed=42)

All
   id  idx  col  size  price  fees  side
0   0    0    0   1.0    1.0   0.0     0
1   1    1    0   1.0    2.0   0.0     1
2   2    3    0   1.0    4.0   0.0     0
3   3    4    0   1.0    5.0   0.0     1
4   4    1    1   1.0    2.0   0.0     1
5   5    3    1   1.0    4.0   0.0     0
6   6    4    1   1.0    5.0   0.0     1
LongOnly
   id  idx  col  size  price  fees  side
0   0    0    0   1.0    1.0   0.0     0
1   1    1    0   1.0    2.0   0.0     1
2   2    3    0   1.0    4.0   0.0     0
3   3    4    0   1.0    5.0   0.0     1
4   4    3    1   1.0    4.0   0.0     0
5   5    4    1   1.0    5.0   0.0     1
ShortOnly
   id  idx  col  size  price  fees  side
0   0    0    0   1.0    1.0   0.0     1
1   1    1    0   1.0    2.0   0.0     0
2   2    3    0   1.0    4.0   0.0     1
3   3    4    0   1.0    5.0   0.0     0
4   4    3    1   1.0    4.0   0.0     1
5   5    4    1   1.0    5.0   0.0     0


In [35]:
test_per_direction(size=pd.Series([np.inf, -np.inf, -np.inf, np.inf, np.inf]), close_first=[[False, True]])

All
__init__() got an unexpected keyword argument 'close_first'
LongOnly
__init__() got an unexpected keyword argument 'close_first'
ShortOnly
__init__() got an unexpected keyword argument 'close_first'


In [36]:
test_per_direction(size=order_size_one * 1000, allow_partial=[[True, False]])
test_per_direction(size=order_size, allow_partial=[[True, False]])

All
   id  idx  col    size  price  fees  side
0   0    0    0   100.0    1.0   0.0     0
1   1    1    0  1000.0    2.0   0.0     1
2   2    3    0   500.0    4.0   0.0     0
3   3    4    0  1000.0    5.0   0.0     1
4   4    1    1  1000.0    2.0   0.0     1
5   5    4    1  1000.0    5.0   0.0     1
LongOnly
   id  idx  col   size  price  fees  side
0   0    0    0  100.0    1.0   0.0     0
1   1    1    0  100.0    2.0   0.0     1
2   2    3    0   50.0    4.0   0.0     0
3   3    4    0   50.0    5.0   0.0     1
ShortOnly
   id  idx  col    size  price  fees  side
0   0    0    0  1000.0    1.0   0.0     1
1   1    1    0   550.0    2.0   0.0     0
2   2    3    0  1000.0    4.0   0.0     1
3   3    4    0   800.0    5.0   0.0     0
4   4    0    1  1000.0    1.0   0.0     1
5   5    3    1  1000.0    4.0   0.0     1
6   6    4    1  1000.0    5.0   0.0     0
All
   id  idx  col   size  price  fees  side
0   0    0    0  100.0    1.0   0.0     0
1   1    1    0  200.0    2.0   0.

In [37]:
test_per_direction(size=order_size_one * 1000, allow_partial=True, raise_reject=True)
test_per_direction(size=order_size_one * 1000, allow_partial=False, raise_reject=True)

All
   id  idx  col    size  price  fees  side
0   0    0    0   100.0    1.0   0.0     0
1   1    1    0  1000.0    2.0   0.0     1
2   2    3    0   500.0    4.0   0.0     0
3   3    4    0  1000.0    5.0   0.0     1
LongOnly
   id  idx  col   size  price  fees  side
0   0    0    0  100.0    1.0   0.0     0
1   1    1    0  100.0    2.0   0.0     1
2   2    3    0   50.0    4.0   0.0     0
3   3    4    0   50.0    5.0   0.0     1
ShortOnly
   id  idx  col    size  price  fees  side
0   0    0    0  1000.0    1.0   0.0     1
1   1    1    0   550.0    2.0   0.0     0
2   2    3    0  1000.0    4.0   0.0     1
3   3    4    0   800.0    5.0   0.0     0
All
Order rejected: Final size is less than requested
LongOnly
Order rejected: Final size is less than requested
ShortOnly
Order rejected: Final size is less than requested


In [38]:
portfolio = vbt.Portfolio.from_orders(price, order_size, log=True, direction='all')

print(portfolio.logs.records)

   id  idx  col  group  cash_now  shares_now  val_price_now  value_now  size  \
0   0    0    0      0     100.0         0.0            1.0      100.0   inf   
1   1    1    0      0       0.0       100.0            2.0      200.0  -inf   
2   2    2    0      0     400.0      -100.0            3.0      100.0   NaN   
3   3    3    0      0     400.0      -100.0            4.0        0.0   inf   
4   4    4    0      0       0.0         0.0            5.0        0.0  -inf   

   size_type  ...   log  new_cash  new_shares  res_size  res_price  res_fees  \
0          0  ...  True       0.0       100.0     100.0        1.0       0.0   
1          0  ...  True     400.0      -100.0     200.0        2.0       0.0   
2          0  ...  True     400.0      -100.0       NaN        NaN       NaN   
3          0  ...  True       0.0         0.0     100.0        4.0       0.0   
4          0  ...  True       0.0         0.0       NaN        NaN       NaN   

   res_side  res_status  res_status_in

In [39]:
portfolio = vbt.Portfolio.from_orders(
    price_wide, order_size, 
    group_by=np.array([0, 0, 1]), direction='all')

print(portfolio.orders.records)

   id  idx  col   size  price  fees  side
0   0    0    0  100.0    1.0   0.0     0
1   1    1    0  200.0    2.0   0.0     1
2   2    3    0  100.0    4.0   0.0     0
3   3    0    1  100.0    1.0   0.0     0
4   4    1    1  200.0    2.0   0.0     1
5   5    3    1  100.0    4.0   0.0     0
6   6    0    2  100.0    1.0   0.0     0
7   7    1    2  200.0    2.0   0.0     1
8   8    3    2  100.0    4.0   0.0     0


In [40]:
portfolio = vbt.Portfolio.from_orders(
    price_wide, order_size, 
    group_by=np.array([0, 0, 1]), cash_sharing=True, direction='all')

print(portfolio.orders.records)
print(portfolio.call_seq)

   id  idx  col   size  price  fees  side
0   0    0    0  100.0    1.0   0.0     0
1   1    1    0  200.0    2.0   0.0     1
2   2    1    1  200.0    2.0   0.0     1
3   3    3    0  200.0    4.0   0.0     0
4   4    4    0  200.0    5.0   0.0     1
5   5    0    2  100.0    1.0   0.0     0
6   6    1    2  200.0    2.0   0.0     1
7   7    3    2  100.0    4.0   0.0     0
            a  b  c
2020-01-01  0  1  0
2020-01-02  0  1  0
2020-01-03  0  1  0
2020-01-04  0  1  0
2020-01-05  0  1  0


In [41]:
portfolio = vbt.Portfolio.from_orders(
    price_wide, order_size, 
    group_by=np.array([0, 0, 1]), cash_sharing=True, direction='all', call_seq='reversed')

print(portfolio.orders.records)
print(portfolio.call_seq)

   id  idx  col   size  price  fees  side
0   0    0    1  100.0    1.0   0.0     0
1   1    1    1  200.0    2.0   0.0     1
2   2    1    0  200.0    2.0   0.0     1
3   3    3    1  200.0    4.0   0.0     0
4   4    4    1  200.0    5.0   0.0     1
5   5    0    2  100.0    1.0   0.0     0
6   6    1    2  200.0    2.0   0.0     1
7   7    3    2  100.0    4.0   0.0     0
            a  b  c
2020-01-01  1  0  0
2020-01-02  1  0  0
2020-01-03  1  0  0
2020-01-04  1  0  0
2020-01-05  1  0  0


In [42]:
portfolio = vbt.Portfolio.from_orders(
    price_wide, order_size, 
    group_by=np.array([0, 0, 1]), cash_sharing=True, direction='all', call_seq='random')

print(portfolio.orders.records)
print(portfolio.call_seq)

portfolio = vbt.Portfolio.from_orders(
    price_wide, order_size, 
    group_by=np.array([0, 0, 1]), cash_sharing=True, direction='all', call_seq='random', seed=42)

print(portfolio.orders.records)
print(portfolio.call_seq)

   id  idx  col   size  price  fees  side
0   0    0    0  100.0    1.0   0.0     0
1   1    1    0  200.0    2.0   0.0     1
2   2    1    1  200.0    2.0   0.0     1
3   3    3    1  200.0    4.0   0.0     0
4   4    0    2  100.0    1.0   0.0     0
5   5    1    2  200.0    2.0   0.0     1
6   6    3    2  100.0    4.0   0.0     0
            a  b  c
2020-01-01  0  1  0
2020-01-02  0  1  0
2020-01-03  0  1  0
2020-01-04  1  0  0
2020-01-05  0  1  0
   id  idx  col   size  price  fees  side
0   0    0    1  100.0    1.0   0.0     0
1   1    1    1  200.0    2.0   0.0     1
2   2    3    1  100.0    4.0   0.0     0
3   3    0    2  100.0    1.0   0.0     0
4   4    1    2  200.0    2.0   0.0     1
5   5    3    2  100.0    4.0   0.0     0
            a  b  c
2020-01-01  1  0  0
2020-01-02  0  1  0
2020-01-03  1  0  0
2020-01-04  1  0  0
2020-01-05  1  0  0


In [43]:
for direction in ('All', 'LongOnly', 'ShortOnly'):
    print(direction)
    portfolio = vbt.Portfolio.from_orders(1., pd.DataFrame([
        [0., 0., np.inf],
        [0., np.inf, -np.inf],
        [np.inf, -np.inf, 0.],
        [-np.inf, 0., np.inf],
        [0., np.inf, -np.inf],
    ]), group_by=np.array([0, 0, 0]), 
        cash_sharing=True, call_seq='auto', direction=direction)

    print(portfolio.orders.records)
    print(portfolio.call_seq)
    print(portfolio.shares())
    print(portfolio.value())
    # notice how shorting with np.inf only accounts for current cash, not for other assets!

All
   id  idx  col    size  price  fees  side
0   0    0    2   100.0    1.0   0.0     0
1   1    1    2   200.0    1.0   0.0     1
2   2    1    1   200.0    1.0   0.0     0
3   3    2    1   400.0    1.0   0.0     1
4   4    2    0   400.0    1.0   0.0     0
5   5    3    0   800.0    1.0   0.0     1
6   6    3    2   800.0    1.0   0.0     0
7   7    4    2  1400.0    1.0   0.0     1
8   8    4    1  1400.0    1.0   0.0     0
   0  1  2
0  0  1  2
1  2  0  1
2  1  2  0
3  0  1  2
4  2  0  1
       0       1      2
0    0.0     0.0  100.0
1    0.0   200.0 -100.0
2  400.0  -200.0 -100.0
3 -400.0  -200.0  700.0
4 -400.0  1200.0 -700.0
0    100.0
1    100.0
2    100.0
3    100.0
4    100.0
dtype: float64
LongOnly
   id  idx  col   size  price  fees  side
0   0    0    2  100.0    1.0   0.0     0
1   1    1    2  100.0    1.0   0.0     1
2   2    1    1  100.0    1.0   0.0     0
3   3    2    1  100.0    1.0   0.0     1
4   4    2    0  100.0    1.0   0.0     0
5   5    3    0  100.0   

In [67]:
test_per_direction(size=[[75., -75.]], size_type=SizeType.TargetShares)

portfolio = vbt.Portfolio.from_orders(
    price, [[75., -75.]], size_type=SizeType.TargetShares, direction='all')
print(portfolio.shares())

portfolio = vbt.Portfolio.from_orders(
    price_wide, 75., size_type=SizeType.TargetShares, 
    group_by=np.array([0, 0, 0]), cash_sharing=True, direction='all')
print(portfolio.orders.records)
print(portfolio.shares())

All
   id  idx  col  size  price  fees  side
0   0    0    0  75.0    1.0   0.0     0
1   1    0    1  75.0    1.0   0.0     1
LongOnly
   id  idx  col  size  price  fees  side
0   0    0    0  75.0    1.0   0.0     0
ShortOnly
   id  idx  col  size  price  fees  side
0   0    0    0  75.0    1.0   0.0     1
               0     1
2020-01-01  75.0 -75.0
2020-01-02  75.0 -75.0
2020-01-03  75.0 -75.0
2020-01-04  75.0 -75.0
2020-01-05  75.0 -75.0
   id  idx  col  size  price  fees  side
0   0    0    0  75.0    1.0   0.0     0
1   1    0    1  25.0    1.0   0.0     0
               a     b    c
2020-01-01  75.0  25.0  0.0
2020-01-02  75.0  25.0  0.0
2020-01-03  75.0  25.0  0.0
2020-01-04  75.0  25.0  0.0
2020-01-05  75.0  25.0  0.0


In [68]:
test_per_direction(size=[[50., -50.]], size_type=SizeType.TargetValue)

portfolio = vbt.Portfolio.from_orders(
    price, [[50., -50.]], size_type=SizeType.TargetValue, direction='all')
print(portfolio.holding_value())

portfolio = vbt.Portfolio.from_orders(
    price_wide, 50., size_type=SizeType.TargetValue, 
    group_by=np.array([0, 0, 0]), cash_sharing=True, direction='all')
print(portfolio.orders.records)
print(portfolio.holding_value(group_by=False))

All
   id  idx  col       size  price  fees  side
0   0    0    0  50.000000    1.0   0.0     0
1   1    1    0  25.000000    2.0   0.0     1
2   2    2    0   8.333333    3.0   0.0     1
3   3    3    0   4.166667    4.0   0.0     1
4   4    4    0   2.500000    5.0   0.0     1
5   5    0    1  50.000000    1.0   0.0     1
6   6    1    1  25.000000    2.0   0.0     0
7   7    2    1   8.333333    3.0   0.0     0
8   8    3    1   4.166667    4.0   0.0     0
9   9    4    1   2.500000    5.0   0.0     0
LongOnly
   id  idx  col       size  price  fees  side
0   0    0    0  50.000000    1.0   0.0     0
1   1    1    0  25.000000    2.0   0.0     1
2   2    2    0   8.333333    3.0   0.0     1
3   3    3    0   4.166667    4.0   0.0     1
4   4    4    0   2.500000    5.0   0.0     1
ShortOnly
   id  idx  col       size  price  fees  side
0   0    0    0  50.000000    1.0   0.0     1
1   1    1    0  25.000000    2.0   0.0     0
2   2    2    0   8.333333    3.0   0.0     0
3   3    3 

In [69]:
test_per_direction(size=[[0.5, -0.5]], size_type=SizeType.TargetPercent)

portfolio = vbt.Portfolio.from_orders(
    price, [[0.5, -0.5]], size_type=SizeType.TargetPercent, direction='all')
print(portfolio.holding_value() / portfolio.value())

portfolio = vbt.Portfolio.from_orders(
    price_wide, 0.5, size_type=SizeType.TargetPercent, 
    group_by=np.array([0, 0, 0]), cash_sharing=True, direction='all')
print(portfolio.orders.records)
print(portfolio.holding_value(group_by=False) / portfolio.value(group_by=False))

All
   id  idx  col       size  price  fees  side
0   0    0    0  50.000000    1.0   0.0     0
1   1    1    0  12.500000    2.0   0.0     1
2   2    2    0   6.250000    3.0   0.0     1
3   3    3    0   3.906250    4.0   0.0     1
4   4    4    0   2.734375    5.0   0.0     1
5   5    0    1  50.000000    1.0   0.0     1
6   6    1    1  37.500000    2.0   0.0     0
7   7    2    1   6.250000    3.0   0.0     0
8   8    3    1   2.343750    4.0   0.0     0
9   9    4    1   1.171875    5.0   0.0     0
LongOnly
   id  idx  col       size  price  fees  side
0   0    0    0  50.000000    1.0   0.0     0
1   1    1    0  12.500000    2.0   0.0     1
2   2    2    0   6.250000    3.0   0.0     1
3   3    3    0   3.906250    4.0   0.0     1
4   4    4    0   2.734375    5.0   0.0     1
ShortOnly
   id  idx  col       size  price  fees  side
0   0    0    0  50.000000    1.0   0.0     1
1   1    1    0  37.500000    2.0   0.0     0
2   2    2    0   6.250000    3.0   0.0     0
3   3    3 

In [70]:
price_one = pd.Series([1., 1., 1., 1., 1.], index=price.index)
target_hold_value = pd.DataFrame({
    'a': [0., 70., 30., 0., 70.],
    'b': [30., 0., 70., 30., 30.],
    'c': [70., 30., 0., 70., 0.]
}, index=price.index)
print(target_hold_value)

               a     b     c
2020-01-01   0.0  30.0  70.0
2020-01-02  70.0   0.0  30.0
2020-01-03  30.0  70.0   0.0
2020-01-04   0.0  30.0  70.0
2020-01-05  70.0  30.0   0.0


In [71]:
portfolio = vbt.Portfolio.from_orders(
    price_one, target_hold_value, size_type=SizeType.TargetValue,
    group_by=np.array([0, 0, 0]), cash_sharing=True,
    call_seq='random', seed=43)

print(portfolio.orders.records)
print(portfolio.share_flow())
print(portfolio.holding_value(group_by=False))  # order fixed -> cannot rebalance properly
print(portfolio.call_seq)

   id  idx  col  size  price  fees  side
0   0    0    1  30.0    1.0   0.0     0
1   1    0    2  70.0    1.0   0.0     0
2   2    1    2  40.0    1.0   0.0     1
3   3    1    1  30.0    1.0   0.0     1
4   4    2    1  70.0    1.0   0.0     0
5   5    2    2  30.0    1.0   0.0     1
6   6    3    2  30.0    1.0   0.0     0
7   7    3    1  40.0    1.0   0.0     1
8   8    4    0  40.0    1.0   0.0     0
9   9    4    2  30.0    1.0   0.0     1
               a     b     c
2020-01-01   0.0  30.0  70.0
2020-01-02   0.0 -30.0 -40.0
2020-01-03   0.0  70.0 -30.0
2020-01-04   0.0 -40.0  30.0
2020-01-05  40.0   0.0 -30.0
               a     b     c
2020-01-01   0.0  30.0  70.0
2020-01-02   0.0   0.0  30.0
2020-01-03   0.0  70.0   0.0
2020-01-04   0.0  30.0  30.0
2020-01-05  40.0  30.0   0.0
            a  b  c
2020-01-01  1  2  0
2020-01-02  0  2  1
2020-01-03  1  0  2
2020-01-04  0  2  1
2020-01-05  1  0  2


In [72]:
portfolio = vbt.Portfolio.from_orders(
    price_one, target_hold_value, size_type=SizeType.TargetValue, 
    group_by=np.array([0, 0, 0]), cash_sharing=True,
    call_seq='auto')

print(portfolio.orders.records)
print(portfolio.share_flow())
print(portfolio.holding_value(group_by=False))  # order dynamic -> can rebalance
print(portfolio.call_seq)

    id  idx  col  size  price  fees  side
0    0    0    1  30.0    1.0   0.0     0
1    1    0    2  70.0    1.0   0.0     0
2    2    1    2  40.0    1.0   0.0     1
3    3    1    1  30.0    1.0   0.0     1
4    4    1    0  70.0    1.0   0.0     0
5    5    2    0  40.0    1.0   0.0     1
6    6    2    2  30.0    1.0   0.0     1
7    7    2    1  70.0    1.0   0.0     0
8    8    3    1  40.0    1.0   0.0     1
9    9    3    0  30.0    1.0   0.0     1
10  10    3    2  70.0    1.0   0.0     0
11  11    4    2  70.0    1.0   0.0     1
12  12    4    0  70.0    1.0   0.0     0
               a     b     c
2020-01-01   0.0  30.0  70.0
2020-01-02  70.0 -30.0 -40.0
2020-01-03 -40.0  70.0 -30.0
2020-01-04 -30.0 -40.0  70.0
2020-01-05  70.0   0.0 -70.0
               a     b     c
2020-01-01   0.0  30.0  70.0
2020-01-02  70.0   0.0  30.0
2020-01-03  30.0  70.0   0.0
2020-01-04   0.0  30.0  70.0
2020-01-05  70.0  30.0   0.0
            a  b  c
2020-01-01  0  1  2
2020-01-02  2  1  0
2020

In [73]:
portfolio = vbt.Portfolio.from_orders(
    price_one, target_hold_value / 100., size_type=SizeType.TargetPercent, 
    group_by=np.array([0, 0, 0]), cash_sharing=True,
    call_seq='auto')

print(portfolio.orders.records)
print(portfolio.share_flow())
print(portfolio.holding_value(group_by=False))
print(portfolio.call_seq)

    id  idx  col  size  price  fees  side
0    0    0    1  30.0    1.0   0.0     0
1    1    0    2  70.0    1.0   0.0     0
2    2    1    2  40.0    1.0   0.0     1
3    3    1    1  30.0    1.0   0.0     1
4    4    1    0  70.0    1.0   0.0     0
5    5    2    0  40.0    1.0   0.0     1
6    6    2    2  30.0    1.0   0.0     1
7    7    2    1  70.0    1.0   0.0     0
8    8    3    1  40.0    1.0   0.0     1
9    9    3    0  30.0    1.0   0.0     1
10  10    3    2  70.0    1.0   0.0     0
11  11    4    2  70.0    1.0   0.0     1
12  12    4    0  70.0    1.0   0.0     0
               a     b     c
2020-01-01   0.0  30.0  70.0
2020-01-02  70.0 -30.0 -40.0
2020-01-03 -40.0  70.0 -30.0
2020-01-04 -30.0 -40.0  70.0
2020-01-05  70.0   0.0 -70.0
               a     b     c
2020-01-01   0.0  30.0  70.0
2020-01-02  70.0   0.0  30.0
2020-01-03  30.0  70.0   0.0
2020-01-04   0.0  30.0  70.0
2020-01-05  70.0  30.0   0.0
            a  b  c
2020-01-01  0  1  2
2020-01-02  2  1  0
2020

## from_order_func

In [44]:
from vectorbt.portfolio.nb import create_order_nb

@njit
def order_func_nb(c, size):
    return create_order_nb(size if c.i % 2 == 0 else -size, c.close[c.i, c.col])

In [46]:
%timeit vbt.Portfolio.from_order_func(\
    big_price, order_func_nb, 1., init_cash=np.inf)
%timeit vbt.Portfolio.from_order_func(\
    big_price, order_func_nb, 1., init_cash=np.inf, row_wise=True)

907 µs ± 3.67 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
935 µs ± 14.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [47]:
%timeit vbt.Portfolio.from_order_func(\
    big_price_wide, order_func_nb, 1., init_cash=np.inf)
%timeit vbt.Portfolio.from_order_func(\
    big_price_wide, order_func_nb, 1., init_cash=np.inf, row_wise=True)

47.3 ms ± 650 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
58.7 ms ± 1.35 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [48]:
%timeit vbt.Portfolio.from_order_func(\
    big_price_wide, order_func_nb, 1., \
    group_by=np.repeat(np.arange(500), 2), init_cash=np.inf)
%timeit vbt.Portfolio.from_order_func(\
    big_price_wide, order_func_nb, 1., \
    group_by=np.repeat(np.arange(500), 2), init_cash=np.inf, row_wise=True)

42.2 ms ± 325 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
48.3 ms ± 621 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [6]:
%timeit vbt.Portfolio.from_order_func(\
    big_price_wide, order_func_nb, 1., \
    group_by=np.repeat(np.arange(500), 2), cash_sharing=True, init_cash=np.inf)
%timeit vbt.Portfolio.from_order_func(\
    big_price_wide, order_func_nb, 1., \
    group_by=np.repeat(np.arange(500), 2), cash_sharing=True, init_cash=np.inf, row_wise=True)

39.6 ms ± 639 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
43.5 ms ± 94.3 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
%timeit vbt.Portfolio.from_order_func(\
    big_price_wide, order_func_nb, 1., init_cash=np.inf, \
    group_by=np.full(1000, 0))
%timeit vbt.Portfolio.from_order_func(\
    big_price_wide, order_func_nb, 1., init_cash=np.inf, \
    group_by=np.full(1000, 0), row_wise=True)

35.2 ms ± 435 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
36 ms ± 101 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
%timeit vbt.Portfolio.from_order_func(\
    big_price_wide, order_func_nb, 1., init_cash=np.inf, \
    group_by=np.full(1000, 0), cash_sharing=True)
%timeit vbt.Portfolio.from_order_func(\
    big_price_wide, order_func_nb, 1., init_cash=np.inf, \
    group_by=np.full(1000, 0), cash_sharing=True, row_wise=True)

36.1 ms ± 103 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
37.4 ms ± 1.53 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [49]:
def test_call_seq(call_seq, **kwargs):
    return vbt.Portfolio.from_order_func(
        price_wide, 
        order_func_nb, 
        1.,
        group_by=np.array([0, 0, 0]),
        call_seq=call_seq, **kwargs
    ).call_seq

def big_test_call_seq(call_seq, **kwargs):
    return vbt.Portfolio.from_order_func(
        big_price_wide, 
        order_func_nb, 
        1.,
        group_by=np.repeat(np.arange(500), 2),
        call_seq=call_seq, **kwargs
    ).call_seq

In [10]:
print(test_call_seq(CallSeqType.Default))
%timeit big_test_call_seq(CallSeqType.Default)

print(test_call_seq(CallSeqType.Reversed))
%timeit big_test_call_seq(CallSeqType.Reversed)

print(test_call_seq(CallSeqType.Random))
%timeit big_test_call_seq(CallSeqType.Random)

            a  b  c
2020-01-01  0  1  2
2020-01-02  0  1  2
2020-01-03  0  1  2
2020-01-04  0  1  2
2020-01-05  0  1  2
46.2 ms ± 838 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
            a  b  c
2020-01-01  2  1  0
2020-01-02  2  1  0
2020-01-03  2  1  0
2020-01-04  2  1  0
2020-01-05  2  1  0
46.4 ms ± 683 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
            a  b  c
2020-01-01  0  1  2
2020-01-02  2  1  0
2020-01-03  1  0  2
2020-01-04  2  0  1
2020-01-05  2  1  0
55.1 ms ± 849 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [37]:
@njit
def segment_prep_func1_nb(c, *args):
    c.call_seq_now[:] = np.arange(c.to_col - c.from_col - 1, -1, -1)
    return ()

@njit
def segment_prep_func2_nb(c, *args):
    c.call_seq_now[:] = np.copy(c.call_seq_now[::-1])
    return ()

@njit
def segment_prep_func3_nb(c, *args):
    call_seq_now = c.call_seq_now
    n_cols = c.to_col - c.from_col
    for k in range(n_cols):
        call_seq_now[k] = n_cols - k - 1
    return ()

@njit
def segment_prep_func4_nb(c, *args):
    np.random.shuffle(c.call_seq_now)
    return ()

In [38]:
print(test_call_seq(CallSeqType.Default, segment_prep_func_nb=segment_prep_func1_nb))
%timeit big_test_call_seq(CallSeqType.Default, segment_prep_func_nb=segment_prep_func1_nb)
%timeit big_test_call_seq(CallSeqType.Default, segment_prep_func_nb=segment_prep_func1_nb, row_wise=True)

print(test_call_seq(CallSeqType.Default, segment_prep_func_nb=segment_prep_func2_nb))
%timeit big_test_call_seq(CallSeqType.Default, segment_prep_func_nb=segment_prep_func2_nb)
%timeit big_test_call_seq(CallSeqType.Default, segment_prep_func_nb=segment_prep_func2_nb, row_wise=True)

print(test_call_seq(CallSeqType.Default, segment_prep_func_nb=segment_prep_func3_nb))
%timeit big_test_call_seq(CallSeqType.Default, segment_prep_func_nb=segment_prep_func3_nb)
%timeit big_test_call_seq(CallSeqType.Default, segment_prep_func_nb=segment_prep_func3_nb, row_wise=True)

print(test_call_seq(CallSeqType.Default, segment_prep_func_nb=segment_prep_func4_nb))
%timeit big_test_call_seq(CallSeqType.Default, segment_prep_func_nb=segment_prep_func4_nb)  # in-place fastest
%timeit big_test_call_seq(CallSeqType.Default, segment_prep_func_nb=segment_prep_func4_nb, row_wise=True)

            a  b  c
2020-01-01  2  1  0
2020-01-02  2  1  0
2020-01-03  2  1  0
2020-01-04  2  1  0
2020-01-05  2  1  0
160 ms ± 44.3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
147 ms ± 5.65 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
            a  b  c
2020-01-01  2  1  0
2020-01-02  2  1  0
2020-01-03  2  1  0
2020-01-04  2  1  0
2020-01-05  2  1  0
163 ms ± 17.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
141 ms ± 1.54 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
            a  b  c
2020-01-01  2  1  0
2020-01-02  2  1  0
2020-01-03  2  1  0
2020-01-04  2  1  0
2020-01-05  2  1  0
53.4 ms ± 7.79 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
89.9 ms ± 26.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
            a  b  c
2020-01-01  0  1  2
2020-01-02  1  0  2
2020-01-03  2  0  1
2020-01-04  0  1  2
2020-01-05  1  2  0
117 ms ± 21.5 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
84.7 ms ± 1.67 ms per loop (mean ±

In [50]:
portfolio = vbt.Portfolio.from_order_func(
    price, order_func_nb, np.inf,
    target_shape=(5,))
print(portfolio.wrapper.columns)
print(portfolio.wrapper.ndim)

portfolio = vbt.Portfolio.from_order_func(
    price, order_func_nb, np.inf,
    target_shape=(5, 1))
print(portfolio.wrapper.columns)
print(portfolio.wrapper.ndim)

portfolio = vbt.Portfolio.from_order_func(
    price, order_func_nb, np.inf,
    target_shape=(5, 1), keys=pd.Index(['first'], name='custom'))
print(portfolio.wrapper.columns)
print(portfolio.wrapper.ndim)

portfolio = vbt.Portfolio.from_order_func(
    price, order_func_nb, np.inf,
    target_shape=(5, 3))
print(portfolio.wrapper.columns)
print(portfolio.wrapper.ndim)

portfolio = vbt.Portfolio.from_order_func(
    price, order_func_nb, np.inf,
    target_shape=(5, 3), keys=pd.Index(['first', 'second', 'third'], name='custom'))
print(portfolio.wrapper.columns)
print(portfolio.wrapper.ndim)

Int64Index([0], dtype='int64')
1
Int64Index([0], dtype='int64', name='iteration_idx')
2
Index(['first'], dtype='object', name='custom')
2
Int64Index([0, 1, 2], dtype='int64', name='iteration_idx')
2
Index(['first', 'second', 'third'], dtype='object', name='custom')
2


In [51]:
portfolio = vbt.Portfolio.from_order_func(price, order_func_nb, np.inf)
print(portfolio.orders.records)

portfolio = vbt.Portfolio.from_order_func(price, order_func_nb, np.inf, row_wise=True)
print(portfolio.orders.records)

   id  idx  col        size  price  fees  side
0   0    0    0  100.000000    1.0   0.0     0
1   1    1    0  200.000000    2.0   0.0     1
2   2    2    0  133.333333    3.0   0.0     0
3   3    3    0   66.666667    4.0   0.0     1
4   4    4    0   53.333333    5.0   0.0     0
   id  idx  col        size  price  fees  side
0   0    0    0  100.000000    1.0   0.0     0
1   1    1    0  200.000000    2.0   0.0     1
2   2    2    0  133.333333    3.0   0.0     0
3   3    3    0   66.666667    4.0   0.0     1
4   4    4    0   53.333333    5.0   0.0     0


In [52]:
portfolio = vbt.Portfolio.from_order_func(price_wide, order_func_nb, np.inf)
print(portfolio.orders.records)

portfolio = vbt.Portfolio.from_order_func(price_wide, order_func_nb, np.inf, row_wise=True)
print(portfolio.orders.records)

    id  idx  col        size  price  fees  side
0    0    0    0  100.000000    1.0   0.0     0
1    1    1    0  200.000000    2.0   0.0     1
2    2    2    0  133.333333    3.0   0.0     0
3    3    3    0   66.666667    4.0   0.0     1
4    4    4    0   53.333333    5.0   0.0     0
5    5    0    1  100.000000    1.0   0.0     0
6    6    1    1  200.000000    2.0   0.0     1
7    7    2    1  133.333333    3.0   0.0     0
8    8    3    1   66.666667    4.0   0.0     1
9    9    4    1   53.333333    5.0   0.0     0
10  10    0    2  100.000000    1.0   0.0     0
11  11    1    2  200.000000    2.0   0.0     1
12  12    2    2  133.333333    3.0   0.0     0
13  13    3    2   66.666667    4.0   0.0     1
14  14    4    2   53.333333    5.0   0.0     0
    id  idx  col        size  price  fees  side
0    0    0    0  100.000000    1.0   0.0     0
1    1    0    1  100.000000    1.0   0.0     0
2    2    0    2  100.000000    1.0   0.0     0
3    3    1    0  200.000000    2.0   0.

In [53]:
portfolio = vbt.Portfolio.from_order_func(
    price_wide, order_func_nb, np.inf,
    group_by=np.array([0, 0, 1]))
print(portfolio.orders.records)

portfolio = vbt.Portfolio.from_order_func(
    price_wide, order_func_nb, np.inf,
    group_by=np.array([0, 0, 1]), row_wise=True)
print(portfolio.orders.records)

    id  idx  col        size  price  fees  side
0    0    0    0  100.000000    1.0   0.0     0
1    1    0    1  100.000000    1.0   0.0     0
2    2    1    0  200.000000    2.0   0.0     1
3    3    1    1  200.000000    2.0   0.0     1
4    4    2    0  133.333333    3.0   0.0     0
5    5    2    1  133.333333    3.0   0.0     0
6    6    3    0   66.666667    4.0   0.0     1
7    7    3    1   66.666667    4.0   0.0     1
8    8    4    0   53.333333    5.0   0.0     0
9    9    4    1   53.333333    5.0   0.0     0
10  10    0    2  100.000000    1.0   0.0     0
11  11    1    2  200.000000    2.0   0.0     1
12  12    2    2  133.333333    3.0   0.0     0
13  13    3    2   66.666667    4.0   0.0     1
14  14    4    2   53.333333    5.0   0.0     0
    id  idx  col        size  price  fees  side
0    0    0    0  100.000000    1.0   0.0     0
1    1    0    1  100.000000    1.0   0.0     0
2    2    0    2  100.000000    1.0   0.0     0
3    3    1    0  200.000000    2.0   0.

In [54]:
portfolio = vbt.Portfolio.from_order_func(
    price_wide, order_func_nb, np.inf,
    group_by=np.array([0, 0, 1]), cash_sharing=True)
print(portfolio.orders.records)

portfolio = vbt.Portfolio.from_order_func(
    price_wide, order_func_nb, np.inf,
    group_by=np.array([0, 0, 1]), cash_sharing=True, row_wise=True)
print(portfolio.orders.records)

    id  idx  col        size  price  fees  side
0    0    0    0  100.000000    1.0   0.0     0
1    1    1    0  200.000000    2.0   0.0     1
2    2    1    1  200.000000    2.0   0.0     1
3    3    2    0  266.666667    3.0   0.0     0
4    4    3    0  333.333333    4.0   0.0     1
5    5    4    0  266.666667    5.0   0.0     0
6    6    0    2  100.000000    1.0   0.0     0
7    7    1    2  200.000000    2.0   0.0     1
8    8    2    2  133.333333    3.0   0.0     0
9    9    3    2   66.666667    4.0   0.0     1
10  10    4    2   53.333333    5.0   0.0     0
    id  idx  col        size  price  fees  side
0    0    0    0  100.000000    1.0   0.0     0
1    1    0    2  100.000000    1.0   0.0     0
2    2    1    0  200.000000    2.0   0.0     1
3    3    1    1  200.000000    2.0   0.0     1
4    4    1    2  200.000000    2.0   0.0     1
5    5    2    0  266.666667    3.0   0.0     0
6    6    2    2  133.333333    3.0   0.0     0
7    7    3    0  333.333333    4.0   0.

In [55]:
@njit
def target_val_segment_prep_func_nb(c, val_price):
    c.last_val_price[c.from_col:c.to_col] = val_price[c.i]
    return ()

@njit
def target_val_order_func_nb(c):
    print(c.last_val_price[c.from_col:c.to_col], 50. / c.close[c.i, c.col] - c.shares_now)
    return create_order_nb(size=50., size_type=SizeType.TargetValue, price=c.close[c.i, c.col])

portfolio = vbt.Portfolio.from_order_func(
    price.iloc[1:], target_val_order_func_nb, 
    segment_prep_func_nb=target_val_segment_prep_func_nb,
    segment_prep_args=(price.values[:-1],)  # lagged valuation price
)
print(portfolio.orders.records)

portfolio = vbt.Portfolio.from_order_func(
    price.iloc[1:], target_val_order_func_nb, 
    segment_prep_func_nb=target_val_segment_prep_func_nb,
    segment_prep_args=(price.values[:-1],), row_wise=True
)
print(portfolio.orders.records)

[1.] 25.0
[2.] -33.33333333333333
[3.] -12.5
[4.] -6.666666666666668
   id  idx  col       size  price  fees  side
0   0    0    0  50.000000    2.0   0.0     0
1   1    1    0  25.000000    3.0   0.0     1
2   2    2    0   8.333333    4.0   0.0     1
3   3    3    0   4.166667    5.0   0.0     1
[1.] 25.0
[2.] -33.33333333333333
[3.] -12.5
[4.] -6.666666666666668
   id  idx  col       size  price  fees  side
0   0    0    0  50.000000    2.0   0.0     0
1   1    1    0  25.000000    3.0   0.0     1
2   2    2    0   8.333333    4.0   0.0     1
3   3    3    0   4.166667    5.0   0.0     1


In [56]:
portfolio = vbt.Portfolio.from_order_func(
    price.iloc[1:], target_val_order_func_nb, 
    segment_prep_func_nb=target_val_segment_prep_func_nb,
    segment_prep_args=(price.values[1:],)  # current order price
)
print(portfolio.orders.records)

portfolio = vbt.Portfolio.from_order_func(
    price.iloc[1:], target_val_order_func_nb, 
    segment_prep_func_nb=target_val_segment_prep_func_nb,
    segment_prep_args=(price.values[1:],), row_wise=True
)
print(portfolio.orders.records)

[2.] 25.0
[3.] -8.333333333333332
[4.] -4.166666666666668
[5.] -2.5
   id  idx  col       size  price  fees  side
0   0    0    0  25.000000    2.0   0.0     0
1   1    1    0   8.333333    3.0   0.0     1
2   2    2    0   4.166667    4.0   0.0     1
3   3    3    0   2.500000    5.0   0.0     1
[2.] 25.0
[3.] -8.333333333333332
[4.] -4.166666666666668
[5.] -2.5
   id  idx  col       size  price  fees  side
0   0    0    0  25.000000    2.0   0.0     0
1   1    1    0   8.333333    3.0   0.0     1
2   2    2    0   4.166667    4.0   0.0     1
3   3    3    0   2.500000    5.0   0.0     1


In [57]:
@njit
def target_pct_segment_prep_func_nb(c, val_price):
    c.last_val_price[c.from_col:c.to_col] = val_price[c.i]
    return ()

@njit
def target_pct_order_func_nb(c):
    print(c.last_val_price[c.from_col:c.to_col], 0.5 * c.value_now / c.close[c.i, c.col] - c.shares_now)
    return create_order_nb(size=0.5, size_type=SizeType.TargetPercent, price=c.close[c.i, c.col])

portfolio = vbt.Portfolio.from_order_func(
    price.iloc[1:], target_pct_order_func_nb, 
    segment_prep_func_nb=target_pct_segment_prep_func_nb,
    segment_prep_args=(price.values[:-1],)  # lagged valuation price
)
print(portfolio.orders.records)

portfolio = vbt.Portfolio.from_order_func(
    price.iloc[1:], target_pct_order_func_nb, 
    segment_prep_func_nb=target_pct_segment_prep_func_nb,
    segment_prep_args=(price.values[:-1],), row_wise=True
)
print(portfolio.orders.records)

[1.] 25.0
[2.] -33.33333333333333
[3.] -6.25
[4.] -7.5
   id  idx  col    size  price  fees  side
0   0    0    0  50.000    2.0   0.0     0
1   1    1    0  25.000    3.0   0.0     1
2   2    3    0   3.125    5.0   0.0     1
[1.] 25.0
[2.] -33.33333333333333
[3.] -6.25
[4.] -7.5
   id  idx  col    size  price  fees  side
0   0    0    0  50.000    2.0   0.0     0
1   1    1    0  25.000    3.0   0.0     1
2   2    3    0   3.125    5.0   0.0     1


In [58]:
portfolio = vbt.Portfolio.from_order_func(
    price.iloc[1:], target_pct_order_func_nb, 
    segment_prep_func_nb=target_pct_segment_prep_func_nb,
    segment_prep_args=(price.values[1:],)  # current order price
)
print(portfolio.orders.records)

portfolio = vbt.Portfolio.from_order_func(
    price.iloc[1:], target_pct_order_func_nb, 
    segment_prep_func_nb=target_pct_segment_prep_func_nb,
    segment_prep_args=(price.values[1:],), row_wise=True
)
print(portfolio.orders.records)

[2.] 25.0
[3.] -4.166666666666668
[4.] -2.604166666666668
[5.] -1.8229166666666643
   id  idx  col       size  price  fees  side
0   0    0    0  25.000000    2.0   0.0     0
1   1    1    0   4.166667    3.0   0.0     1
2   2    2    0   2.604167    4.0   0.0     1
3   3    3    0   1.822917    5.0   0.0     1
[2.] 25.0
[3.] -4.166666666666668
[4.] -2.604166666666668
[5.] -1.8229166666666643
   id  idx  col       size  price  fees  side
0   0    0    0  25.000000    2.0   0.0     0
1   1    1    0   4.166667    3.0   0.0     1
2   2    2    0   2.604167    4.0   0.0     1
3   3    3    0   1.822917    5.0   0.0     1


In [59]:
portfolio = vbt.Portfolio.from_order_func(
    price, order_func_nb, 1.,
    active_mask=pd.Series([True, False, True, False, True]))
print(portfolio.orders.records)

portfolio = vbt.Portfolio.from_order_func(
    price, order_func_nb, 1.,
    active_mask=pd.Series([True, False, True, False, True]), row_wise=True)
print(portfolio.orders.records)

   id  idx  col  size  price  fees  side
0   0    0    0   1.0    1.0   0.0     0
1   1    2    0   1.0    3.0   0.0     0
2   2    4    0   1.0    5.0   0.0     0
   id  idx  col  size  price  fees  side
0   0    0    0   1.0    1.0   0.0     0
1   1    2    0   1.0    3.0   0.0     0
2   2    4    0   1.0    5.0   0.0     0


In [60]:
@njit
def prep_func_nb(c, sim_i):
    print('preparing simulation')
    sim_i[0] += 1
    return sim_i,

@njit
def group_prep_func_nb(c, sim_i, group_i):
    print('\tpreparing group', c.group)
    group_i[0] += 1
    return sim_i, group_i

@njit
def segment_prep_func_nb(c, sim_i, group_i, segment_i):
    print('\t\tpreparing segment', c.i, '(row)')
    segment_i[0] += 1
    return sim_i, group_i, segment_i

@njit
def order_func_nb(c, sim_i, group_i, segment_i, order_i):
    print('\t\t\trunning order', c.call_idx, 'at column', c.col)
    order_i[0] += 1
    return NoOrder

sim_i = np.array([0])
group_i = np.array([0])
segment_i = np.array([0])
order_i = np.array([0])
portfolio = vbt.Portfolio.from_order_func(
    price_wide, order_func_nb, order_i, 
    group_by=np.array([0, 0, 1]), 
    prep_func_nb=prep_func_nb, prep_args=(sim_i,),
    group_prep_func_nb=group_prep_func_nb, group_prep_args=(group_i,),
    segment_prep_func_nb=segment_prep_func_nb, segment_prep_args=(segment_i,)
)
print(sim_i, group_i, segment_i, order_i)

preparing simulation
	preparing group 0
		preparing segment 0 (row)
			running order 0 at column 0
			running order 1 at column 1
		preparing segment 1 (row)
			running order 0 at column 0
			running order 1 at column 1
		preparing segment 2 (row)
			running order 0 at column 0
			running order 1 at column 1
		preparing segment 3 (row)
			running order 0 at column 0
			running order 1 at column 1
		preparing segment 4 (row)
			running order 0 at column 0
			running order 1 at column 1
	preparing group 1
		preparing segment 0 (row)
			running order 0 at column 2
		preparing segment 1 (row)
			running order 0 at column 2
		preparing segment 2 (row)
			running order 0 at column 2
		preparing segment 3 (row)
			running order 0 at column 2
		preparing segment 4 (row)
			running order 0 at column 2
[1] [2] [10] [15]


In [61]:
@njit
def prep_func_nb(c, sim_i):
    print('preparing simulation')
    sim_i[0] += 1
    return sim_i,

@njit
def row_prep_func_nb(c, sim_i, row_i):
    print('\tpreparing row', c.i)
    row_i[0] += 1
    return sim_i, row_i

@njit
def segment_prep_func_nb(c, sim_i, row_i, segment_i):
    print('\t\tpreparing segment', c.group, '(group)')
    segment_i[0] += 1
    return sim_i, row_i, segment_i

@njit
def order_func_nb(c, sim_i, row_i, segment_i, order_i):
    print('\t\t\trunning order', c.call_idx, 'at column', c.col)
    order_i[0] += 1
    return NoOrder

sim_i = np.array([0])
row_i = np.array([0])
segment_i = np.array([0])
order_i = np.array([0])

portfolio = vbt.Portfolio.from_order_func(
    price_wide, order_func_nb, order_i, 
    group_by=np.array([0, 0, 1]), 
    prep_func_nb=prep_func_nb, prep_args=(sim_i,),
    row_prep_func_nb=row_prep_func_nb, row_prep_args=(row_i,),
    segment_prep_func_nb=segment_prep_func_nb, segment_prep_args=(segment_i,),
    row_wise=True
)
print(sim_i, row_i, segment_i, order_i)

preparing simulation
	preparing row 0
		preparing segment 0 (group)
			running order 0 at column 0
			running order 1 at column 1
		preparing segment 1 (group)
			running order 0 at column 2
	preparing row 1
		preparing segment 0 (group)
			running order 0 at column 0
			running order 1 at column 1
		preparing segment 1 (group)
			running order 0 at column 2
	preparing row 2
		preparing segment 0 (group)
			running order 0 at column 0
			running order 1 at column 1
		preparing segment 1 (group)
			running order 0 at column 2
	preparing row 3
		preparing segment 0 (group)
			running order 0 at column 0
			running order 1 at column 1
		preparing segment 1 (group)
			running order 0 at column 2
	preparing row 4
		preparing segment 0 (group)
			running order 0 at column 0
			running order 1 at column 1
		preparing segment 1 (group)
			running order 0 at column 2
[1] [5] [10] [15]


## methods and properties

In [8]:
price_na = pd.DataFrame({
    'a': [np.nan, 2., 3., 4., 5.],
    'b': [1., 2., np.nan, 4., 5.],
    'c': [1., 2., 3., 4., np.nan]
}, index=price.index)
order_size = pd.Series([1., 0.1, -1., -0.1, 1.])
directions = ['longonly', 'shortonly', 'all']
group_by = pd.Index(['first', 'first', 'second'], name='group')

In [9]:
portfolio = vbt.Portfolio.from_orders(
    price_na, order_size, size_type='shares', direction=directions,
    fees=0.01, fixed_fees=0.1, slippage=0.01, log=True,
    call_seq='reversed', group_by=None,
    init_cash=[100., 100., 100.], freq='1D'
)  # independent
print(portfolio.orders.records)

portfolio_grouped = vbt.Portfolio.from_orders(
    price_na, order_size, size_type='shares', direction=directions,
    fees=0.01, fixed_fees=0.1, slippage=0.01, log=True,
    call_seq='reversed', group_by=group_by, cash_sharing=False,
    init_cash=[100., 100., 100.], freq='1D'
)  # grouped
print(portfolio_grouped.orders.records)

portfolio_shared = vbt.Portfolio.from_orders(
    price_na, order_size, size_type='shares', direction=directions,
    fees=0.01, fixed_fees=0.1, slippage=0.01, log=True,
    call_seq='reversed', group_by=group_by, cash_sharing=True,
    init_cash=[200., 100.], freq='1D'
)  # shared
print(portfolio_shared.orders.records)

    id  idx  col  size  price     fees  side
0    0    1    0   0.1   2.02  0.10202     0
1    1    2    0   0.1   2.97  0.10297     1
2    2    4    0   1.0   5.05  0.15050     0
3    3    0    1   1.0   0.99  0.10990     1
4    4    1    1   0.1   1.98  0.10198     1
5    5    3    1   0.1   4.04  0.10404     0
6    6    4    1   1.0   4.95  0.14950     1
7    7    0    2   1.0   1.01  0.11010     0
8    8    1    2   0.1   2.02  0.10202     0
9    9    2    2   1.0   2.97  0.12970     1
10  10    3    2   0.1   3.96  0.10396     1
    id  idx  col  size  price     fees  side
0    0    1    0   0.1   2.02  0.10202     0
1    1    2    0   0.1   2.97  0.10297     1
2    2    4    0   1.0   5.05  0.15050     0
3    3    0    1   1.0   0.99  0.10990     1
4    4    1    1   0.1   1.98  0.10198     1
5    5    3    1   0.1   4.04  0.10404     0
6    6    4    1   1.0   4.95  0.14950     1
7    7    0    2   1.0   1.01  0.11010     0
8    8    1    2   0.1   2.02  0.10202     0
9    9    

In [10]:
big_group_by = np.repeat(np.arange(500), 2)
big_portfolio = vbt.Portfolio.from_orders(
    big_price_wide, big_order_size)
print(big_portfolio.orders.values.shape)

big_portfolio_grouped = vbt.Portfolio.from_orders(
    big_price_wide, big_order_size, group_by=big_group_by)
print(big_portfolio_grouped.orders.values.shape)

big_portfolio_shared = vbt.Portfolio.from_orders(
    big_price_wide, big_order_size, group_by=big_group_by, cash_sharing=True)
print(big_portfolio_shared.orders.values.shape)

(1000000,)
(1000000,)
(1000000,)


In [11]:
print(portfolio.wrapper.index)
print(portfolio.wrapper.columns)
print(portfolio.wrapper.ndim)
print(portfolio.wrapper.grouper.group_by)
print(portfolio.wrapper.grouper.allow_enable)
print(portfolio.wrapper.grouper.allow_disable)
print(portfolio.wrapper.grouper.allow_modify)

print(portfolio_grouped.wrapper.index)
print(portfolio_grouped.wrapper.columns)
print(portfolio_grouped.wrapper.ndim)
print(portfolio_grouped.wrapper.grouper.group_by)
print(portfolio_grouped.wrapper.grouper.allow_enable)
print(portfolio_grouped.wrapper.grouper.allow_disable)
print(portfolio_grouped.wrapper.grouper.allow_modify)

print(portfolio_shared.wrapper.index)
print(portfolio_shared.wrapper.columns)
print(portfolio_shared.wrapper.ndim)
print(portfolio_shared.wrapper.grouper.group_by)
print(portfolio_shared.wrapper.grouper.allow_enable)
print(portfolio_shared.wrapper.grouper.allow_disable)
print(portfolio_shared.wrapper.grouper.allow_modify)

DatetimeIndex(['2020-01-01', '2020-01-02', '2020-01-03', '2020-01-04',
               '2020-01-05'],
              dtype='datetime64[ns]', freq=None)
Index(['a', 'b', 'c'], dtype='object')
2
None
True
True
True
DatetimeIndex(['2020-01-01', '2020-01-02', '2020-01-03', '2020-01-04',
               '2020-01-05'],
              dtype='datetime64[ns]', freq=None)
Index(['a', 'b', 'c'], dtype='object')
2
Index(['first', 'first', 'second'], dtype='object', name='group')
True
True
True
DatetimeIndex(['2020-01-01', '2020-01-02', '2020-01-03', '2020-01-04',
               '2020-01-05'],
              dtype='datetime64[ns]', freq=None)
Index(['a', 'b', 'c'], dtype='object')
2
Index(['first', 'first', 'second'], dtype='object', name='group')
False
True
False


In [54]:
print(portfolio.orders.records)
print(portfolio.orders.count())
print(portfolio.logs.count())
print(portfolio.close)
print(portfolio.init_cash)
print(portfolio.value())
print(portfolio.final_value())

    id  idx  col  size  price     fees  side
0    0    1    0   0.1   2.02  0.10202     0
1    1    2    0   0.1   2.97  0.10297     1
2    2    4    0   1.0   5.05  0.15050     0
3    3    0    1   1.0   0.99  0.10990     1
4    4    1    1   0.1   1.98  0.10198     1
5    5    3    1   0.1   4.04  0.10404     0
6    6    4    1   1.0   4.95  0.14950     1
7    7    0    2   1.0   1.01  0.11010     0
8    8    1    2   0.1   2.02  0.10202     0
9    9    2    2   1.0   2.97  0.12970     1
10  10    3    2   0.1   3.96  0.10396     1
a    3
b    4
c    4
Name: count, dtype: int64
a    5
b    5
c    5
Name: count, dtype: int64
              a    b    c
2020-01-01  NaN  1.0  1.0
2020-01-02  2.0  2.0  2.0
2020-01-03  3.0  NaN  3.0
2020-01-04  4.0  4.0  4.0
2020-01-05  5.0  5.0  NaN
a    100.0
b    100.0
c    100.0
Name: init_cash, dtype: float64
                    a         b          c
2020-01-01  100.00000  99.88010   99.87990
2020-01-02   99.89598  98.77612  100.77588
2020-01-03   99.

In [55]:
print(portfolio['a'].orders.records)
print(portfolio['a'].orders.count())
print(portfolio['a'].logs.count())
print(portfolio['a'].close)
print(portfolio['a'].init_cash)
print(portfolio['a'].value())
print(portfolio['a'].final_value())

   id  idx  col  size  price     fees  side
0   0    1    0   0.1   2.02  0.10202     0
1   1    2    0   0.1   2.97  0.10297     1
2   2    4    0   1.0   5.05  0.15050     0
3
5
2020-01-01    NaN
2020-01-02    2.0
2020-01-03    3.0
2020-01-04    4.0
2020-01-05    5.0
Name: a, dtype: float64
100.0
2020-01-01    100.00000
2020-01-02     99.89598
2020-01-03     99.89001
2020-01-04     99.89001
2020-01-05     99.68951
Name: a, dtype: float64
99.68951


In [56]:
print(portfolio['c'].orders.records)
print(portfolio['c'].orders.count())
print(portfolio['c'].logs.count())
print(portfolio['c'].close)
print(portfolio['c'].init_cash)
print(portfolio['c'].value())
print(portfolio['c'].final_value())

   id  idx  col  size  price     fees  side
0   7    0    0   1.0   1.01  0.11010     0
1   8    1    0   0.1   2.02  0.10202     0
2   9    2    0   1.0   2.97  0.12970     1
3  10    3    0   0.1   3.96  0.10396     1
4
5
2020-01-01    1.0
2020-01-02    2.0
2020-01-03    3.0
2020-01-04    4.0
2020-01-05    NaN
Name: c, dtype: float64
100.0
2020-01-01     99.87990
2020-01-02    100.77588
2020-01-03    101.71618
2020-01-04    101.70822
2020-01-05    101.70822
Name: c, dtype: float64
101.70822


In [57]:
print(portfolio[['c']].orders.records)
print(portfolio[['c']].orders.count())
print(portfolio[['c']].logs.count())
print(portfolio[['c']].close)
print(portfolio[['c']].init_cash)
print(portfolio[['c']].value())
print(portfolio[['c']].final_value())

   id  idx  col  size  price     fees  side
0   7    0    0   1.0   1.01  0.11010     0
1   8    1    0   0.1   2.02  0.10202     0
2   9    2    0   1.0   2.97  0.12970     1
3  10    3    0   0.1   3.96  0.10396     1
c    4
Name: count, dtype: int64
c    5
Name: count, dtype: int64
              c
2020-01-01  1.0
2020-01-02  2.0
2020-01-03  3.0
2020-01-04  4.0
2020-01-05  NaN
c    100.0
Name: init_cash, dtype: float64
                    c
2020-01-01   99.87990
2020-01-02  100.77588
2020-01-03  101.71618
2020-01-04  101.70822
2020-01-05  101.70822
c    101.70822
Name: final_value, dtype: float64


In [58]:
print(portfolio_grouped.orders.records)
print(portfolio_grouped.orders.count())
print(portfolio_grouped.logs.count())
print(portfolio_grouped.close)
print(portfolio_grouped.init_cash)
print(portfolio_grouped.value())
print(portfolio_grouped.final_value())

    id  idx  col  size  price     fees  side
0    0    1    0   0.1   2.02  0.10202     0
1    1    2    0   0.1   2.97  0.10297     1
2    2    4    0   1.0   5.05  0.15050     0
3    3    0    1   1.0   0.99  0.10990     1
4    4    1    1   0.1   1.98  0.10198     1
5    5    3    1   0.1   4.04  0.10404     0
6    6    4    1   1.0   4.95  0.14950     1
7    7    0    2   1.0   1.01  0.11010     0
8    8    1    2   0.1   2.02  0.10202     0
9    9    2    2   1.0   2.97  0.12970     1
10  10    3    2   0.1   3.96  0.10396     1
group
first     7
second    4
Name: count, dtype: int64
group
first     10
second     5
Name: count, dtype: int64
              a    b    c
2020-01-01  NaN  1.0  1.0
2020-01-02  2.0  2.0  2.0
2020-01-03  3.0  NaN  3.0
2020-01-04  4.0  4.0  4.0
2020-01-05  5.0  5.0  NaN
group
first     200.0
second    100.0
Name: init_cash, dtype: float64
group           first     second
2020-01-01  199.88010   99.87990
2020-01-02  198.67210  100.77588
2020-01-03        NaN

In [59]:
print(portfolio_grouped['first'].orders.records)
print(portfolio_grouped['first'].orders.count())
print(portfolio_grouped['first'].logs.count())
print(portfolio_grouped['first'].close)
print(portfolio_grouped['first'].init_cash)
print(portfolio_grouped['first'].value())
print(portfolio_grouped['first'].final_value())

   id  idx  col  size  price     fees  side
0   0    1    0   0.1   2.02  0.10202     0
1   1    2    0   0.1   2.97  0.10297     1
2   2    4    0   1.0   5.05  0.15050     0
3   3    0    1   1.0   0.99  0.10990     1
4   4    1    1   0.1   1.98  0.10198     1
5   5    3    1   0.1   4.04  0.10404     0
6   6    4    1   1.0   4.95  0.14950     1
7
10
              a    b
2020-01-01  NaN  1.0
2020-01-02  2.0  2.0
2020-01-03  3.0  NaN
2020-01-04  4.0  4.0
2020-01-05  5.0  5.0
200.0
2020-01-01    199.88010
2020-01-02    198.67210
2020-01-03          NaN
2020-01-04    196.35809
2020-01-05    194.95809
Name: first, dtype: float64
194.95809


In [60]:
print(portfolio_grouped['second'].orders.records)
print(portfolio_grouped['second'].orders.count())
print(portfolio_grouped['second'].logs.count())
print(portfolio_grouped['second'].close)
print(portfolio_grouped['second'].init_cash)
print(portfolio_grouped['second'].value())
print(portfolio_grouped['second'].final_value())

   id  idx  col  size  price     fees  side
0   7    0    0   1.0   1.01  0.11010     0
1   8    1    0   0.1   2.02  0.10202     0
2   9    2    0   1.0   2.97  0.12970     1
3  10    3    0   0.1   3.96  0.10396     1
4
5
2020-01-01    1.0
2020-01-02    2.0
2020-01-03    3.0
2020-01-04    4.0
2020-01-05    NaN
Name: c, dtype: float64
100.0
2020-01-01     99.87990
2020-01-02    100.77588
2020-01-03    101.71618
2020-01-04    101.70822
2020-01-05    101.70822
Name: second, dtype: float64
101.70822


In [61]:
print(portfolio_grouped[['second']].orders.records)
print(portfolio_grouped[['second']].orders.count())
print(portfolio_grouped[['second']].logs.count())
print(portfolio_grouped[['second']].close)
print(portfolio_grouped[['second']].init_cash)
print(portfolio_grouped[['second']].value())
print(portfolio_grouped[['second']].final_value())

   id  idx  col  size  price     fees  side
0   7    0    0   1.0   1.01  0.11010     0
1   8    1    0   0.1   2.02  0.10202     0
2   9    2    0   1.0   2.97  0.12970     1
3  10    3    0   0.1   3.96  0.10396     1
group
second    4
Name: count, dtype: int64
group
second    5
Name: count, dtype: int64
              c
2020-01-01  1.0
2020-01-02  2.0
2020-01-03  3.0
2020-01-04  4.0
2020-01-05  NaN
group
second    100.0
Name: init_cash, dtype: float64
group          second
2020-01-01   99.87990
2020-01-02  100.77588
2020-01-03  101.71618
2020-01-04  101.70822
2020-01-05  101.70822
group
second    101.70822
Name: final_value, dtype: float64


In [62]:
%timeit big_portfolio.iloc[0]
%timeit big_portfolio.iloc[:]
%timeit big_portfolio_grouped.iloc[0]
%timeit big_portfolio_grouped.iloc[:]
%timeit big_portfolio_shared.iloc[0]
%timeit big_portfolio_shared.iloc[:]

3.42 ms ± 283 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
The slowest run took 17.43 times longer than the fastest. This could mean that an intermediate result is being cached.
111 ms ± 89.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
3.37 ms ± 46.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
14.3 ms ± 1.99 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
6.41 ms ± 139 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
18.1 ms ± 366 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [63]:
print(portfolio.cash_sharing)
print(portfolio_grouped.cash_sharing)
print(portfolio_shared.cash_sharing)

False
False
True


In [64]:
print(portfolio.call_seq)
print(portfolio_grouped.call_seq)
print(portfolio_shared.call_seq)

            a  b  c
2020-01-01  0  0  0
2020-01-02  0  0  0
2020-01-03  0  0  0
2020-01-04  0  0  0
2020-01-05  0  0  0
            a  b  c
2020-01-01  1  0  0
2020-01-02  1  0  0
2020-01-03  1  0  0
2020-01-04  1  0  0
2020-01-05  1  0  0
            a  b  c
2020-01-01  1  0  0
2020-01-02  1  0  0
2020-01-03  1  0  0
2020-01-04  1  0  0
2020-01-05  1  0  0


In [65]:
print(portfolio.orders.count())
print(portfolio_grouped.get_orders(group_by=False).count())
print(portfolio_shared.get_orders(group_by=False).count())

print(portfolio.get_orders(group_by=group_by).count())
print(portfolio_grouped.orders.count())
print(portfolio_shared.orders.count())

a    3
b    4
c    4
Name: count, dtype: int64
a    3
b    4
c    4
Name: count, dtype: int64
a    3
b    4
c    4
Name: count, dtype: int64
group
first     7
second    4
Name: count, dtype: int64
group
first     7
second    4
Name: count, dtype: int64
group
first     7
second    4
Name: count, dtype: int64


In [66]:
print(portfolio.logs.count())
print(portfolio_grouped.get_logs(group_by=False).count())
print(portfolio_shared.get_logs(group_by=False).count())

print(portfolio.get_logs(group_by=group_by).count())
print(portfolio_grouped.logs.count())
print(portfolio_shared.logs.count())

a    5
b    5
c    5
Name: count, dtype: int64
a    5
b    5
c    5
Name: count, dtype: int64
a    5
b    5
c    5
Name: count, dtype: int64
group
first     10
second     5
Name: count, dtype: int64
group
first     10
second     5
Name: count, dtype: int64
group
first     10
second     5
Name: count, dtype: int64


In [67]:
print(portfolio.trades.count())
print(portfolio_grouped.get_trades(group_by=False).count())
print(portfolio_shared.get_trades(group_by=False).count())

print(portfolio.get_trades(group_by=group_by).count())
print(portfolio_grouped.trades.count())
print(portfolio_shared.trades.count())

a    2
b    2
c    2
Name: count, dtype: int64
a    2
b    2
c    2
Name: count, dtype: int64
a    2
b    2
c    2
Name: count, dtype: int64
group
first     4
second    2
Name: count, dtype: int64
group
first     4
second    2
Name: count, dtype: int64
group
first     4
second    2
Name: count, dtype: int64


In [68]:
%timeit big_portfolio.trades

32.7 ms ± 9.97 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [69]:
print(portfolio.positions.count())
print(portfolio_grouped.get_positions(group_by=False).count())
print(portfolio_shared.get_positions(group_by=False).count())

print(portfolio.get_positions(group_by=group_by).count())
print(portfolio_grouped.positions.count())
print(portfolio_shared.positions.count())

a    2
b    1
c    1
Name: count, dtype: int64
a    2
b    1
c    1
Name: count, dtype: int64
a    2
b    1
c    1
Name: count, dtype: int64
group
first     3
second    1
Name: count, dtype: int64
group
first     3
second    1
Name: count, dtype: int64
group
first     3
second    1
Name: count, dtype: int64


In [70]:
%timeit big_portfolio.positions

26.9 ms ± 1.16 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [71]:
print(portfolio.drawdowns.count())
print(portfolio_grouped.get_drawdowns(group_by=False).count())
print(portfolio_shared.get_drawdowns(group_by=False).count())

print(portfolio.get_drawdowns(group_by=group_by).count())
print(portfolio_grouped.drawdowns.count())
print(portfolio_shared.drawdowns.count())

a    1
b    1
c    1
Name: count, dtype: int64
a    1
b    1
c    1
Name: count, dtype: int64
a    1
b    1
c    1
Name: count, dtype: int64
group
first     1
second    1
Name: count, dtype: int64
group
first     1
second    1
Name: count, dtype: int64
group
first     1
second    1
Name: count, dtype: int64


In [72]:
%timeit big_portfolio.drawdowns

39.9 ms ± 907 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [73]:
print(portfolio.close)
print(portfolio_grouped.close)
print(portfolio_shared.close)

              a    b    c
2020-01-01  NaN  1.0  1.0
2020-01-02  2.0  2.0  2.0
2020-01-03  3.0  NaN  3.0
2020-01-04  4.0  4.0  4.0
2020-01-05  5.0  5.0  NaN
              a    b    c
2020-01-01  NaN  1.0  1.0
2020-01-02  2.0  2.0  2.0
2020-01-03  3.0  NaN  3.0
2020-01-04  4.0  4.0  4.0
2020-01-05  5.0  5.0  NaN
              a    b    c
2020-01-01  NaN  1.0  1.0
2020-01-02  2.0  2.0  2.0
2020-01-03  3.0  NaN  3.0
2020-01-04  4.0  4.0  4.0
2020-01-05  5.0  5.0  NaN


In [74]:
print(portfolio.fill_close(ffill=False, bfill=False))
print(portfolio.fill_close(ffill=True, bfill=False))
print(portfolio.fill_close(ffill=False, bfill=True))
print(portfolio.fill_close(ffill=True, bfill=True))

              a    b    c
2020-01-01  NaN  1.0  1.0
2020-01-02  2.0  2.0  2.0
2020-01-03  3.0  NaN  3.0
2020-01-04  4.0  4.0  4.0
2020-01-05  5.0  5.0  NaN
              a    b    c
2020-01-01  NaN  1.0  1.0
2020-01-02  2.0  2.0  2.0
2020-01-03  3.0  2.0  3.0
2020-01-04  4.0  4.0  4.0
2020-01-05  5.0  5.0  4.0
              a    b    c
2020-01-01  2.0  1.0  1.0
2020-01-02  2.0  2.0  2.0
2020-01-03  3.0  4.0  3.0
2020-01-04  4.0  4.0  4.0
2020-01-05  5.0  5.0  NaN
              a    b    c
2020-01-01  2.0  1.0  1.0
2020-01-02  2.0  2.0  2.0
2020-01-03  3.0  2.0  3.0
2020-01-04  4.0  4.0  4.0
2020-01-05  5.0  5.0  4.0


In [75]:
%timeit big_portfolio.fill_close()

67.6 µs ± 1.39 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [76]:
print(portfolio.share_flow(direction='longonly'))
print(portfolio.share_flow(direction='shortonly'))

print(portfolio.share_flow())
print(portfolio_grouped.share_flow())
print(portfolio_shared.share_flow())

              a    b    c
2020-01-01  0.0  0.0  1.0
2020-01-02  0.1  0.0  0.1
2020-01-03 -0.1  0.0 -1.0
2020-01-04  0.0  0.0 -0.1
2020-01-05  1.0  0.0  0.0
              a    b    c
2020-01-01  0.0  1.0  0.0
2020-01-02  0.0  0.1  0.0
2020-01-03  0.0  0.0  0.0
2020-01-04  0.0 -0.1  0.0
2020-01-05  0.0  1.0  0.0
              a    b    c
2020-01-01  0.0 -1.0  1.0
2020-01-02  0.1 -0.1  0.1
2020-01-03 -0.1  0.0 -1.0
2020-01-04  0.0  0.1 -0.1
2020-01-05  1.0 -1.0  0.0
              a    b    c
2020-01-01  0.0 -1.0  1.0
2020-01-02  0.1 -0.1  0.1
2020-01-03 -0.1  0.0 -1.0
2020-01-04  0.0  0.1 -0.1
2020-01-05  1.0 -1.0  0.0
              a    b    c
2020-01-01  0.0 -1.0  1.0
2020-01-02  0.1 -0.1  0.1
2020-01-03 -0.1  0.0 -1.0
2020-01-04  0.0  0.1 -0.1
2020-01-05  1.0 -1.0  0.0


In [77]:
%timeit big_portfolio.share_flow()
%timeit big_portfolio_grouped.share_flow()
%timeit big_portfolio_shared.share_flow()

11 ms ± 539 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
10.6 ms ± 276 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
10.5 ms ± 148 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [78]:
print(portfolio.shares(direction='longonly'))
print(portfolio.shares(direction='shortonly'))

print(portfolio.shares())
print(portfolio_grouped.shares())
print(portfolio_shared.shares())

              a    b    c
2020-01-01  0.0  0.0  1.0
2020-01-02  0.1  0.0  1.1
2020-01-03  0.0  0.0  0.1
2020-01-04  0.0  0.0  0.0
2020-01-05  1.0  0.0  0.0
              a    b    c
2020-01-01  0.0  1.0  0.0
2020-01-02  0.0  1.1  0.0
2020-01-03  0.0  1.1  0.0
2020-01-04  0.0  1.0  0.0
2020-01-05  0.0  2.0  0.0
              a    b    c
2020-01-01  0.0 -1.0  1.0
2020-01-02  0.1 -1.1  1.1
2020-01-03  0.0 -1.1  0.1
2020-01-04  0.0 -1.0  0.0
2020-01-05  1.0 -2.0  0.0
              a    b    c
2020-01-01  0.0 -1.0  1.0
2020-01-02  0.1 -1.1  1.1
2020-01-03  0.0 -1.1  0.1
2020-01-04  0.0 -1.0  0.0
2020-01-05  1.0 -2.0  0.0
              a    b    c
2020-01-01  0.0 -1.0  1.0
2020-01-02  0.1 -1.1  1.1
2020-01-03  0.0 -1.1  0.1
2020-01-04  0.0 -1.0  0.0
2020-01-05  1.0 -2.0  0.0


In [79]:
%timeit big_portfolio.shares()
%timeit big_portfolio_grouped.shares()
%timeit big_portfolio_shared.shares()

14.4 ms ± 539 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
14.6 ms ± 265 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
14 ms ± 50.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [80]:
print(portfolio.pos_mask(direction='longonly'))
print(portfolio.pos_mask(direction='shortonly'))

print(portfolio.pos_mask())
print(portfolio_grouped.pos_mask(group_by=False))
print(portfolio_shared.pos_mask(group_by=False))

print(portfolio.pos_mask(group_by=group_by))
print(portfolio_grouped.pos_mask())
print(portfolio_shared.pos_mask())

                a      b      c
2020-01-01  False  False   True
2020-01-02   True  False   True
2020-01-03  False  False   True
2020-01-04  False  False  False
2020-01-05   True  False  False
                a     b      c
2020-01-01  False  True  False
2020-01-02  False  True  False
2020-01-03  False  True  False
2020-01-04  False  True  False
2020-01-05  False  True  False
                a     b      c
2020-01-01  False  True   True
2020-01-02   True  True   True
2020-01-03  False  True   True
2020-01-04  False  True  False
2020-01-05   True  True  False
                a     b      c
2020-01-01  False  True   True
2020-01-02   True  True   True
2020-01-03  False  True   True
2020-01-04  False  True  False
2020-01-05   True  True  False
                a     b      c
2020-01-01  False  True   True
2020-01-02   True  True   True
2020-01-03  False  True   True
2020-01-04  False  True  False
2020-01-05   True  True  False
group       first  second
2020-01-01   True    True
2020-01-02  

In [81]:
%timeit big_portfolio.pos_mask()
%timeit big_portfolio_grouped.pos_mask(group_by=False)
%timeit big_portfolio_shared.pos_mask(group_by=False)

%timeit big_portfolio.pos_mask(group_by=big_group_by)
%timeit big_portfolio_grouped.pos_mask()
%timeit big_portfolio_shared.pos_mask()

14.8 ms ± 113 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
15.1 ms ± 462 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
15.6 ms ± 299 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
31.4 ms ± 275 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
31 ms ± 198 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
32 ms ± 208 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [82]:
print(portfolio.pos_coverage(direction='longonly'))
print(portfolio.pos_coverage(direction='shortonly'))

print(portfolio.pos_coverage())
print(portfolio_grouped.pos_coverage(group_by=False))
print(portfolio_shared.pos_coverage(group_by=False))

print(portfolio.pos_coverage(group_by=group_by))
print(portfolio_grouped.pos_coverage())
print(portfolio_shared.pos_coverage())

a    0.4
b    0.0
c    0.6
Name: pos_coverage, dtype: float64
a    0.0
b    1.0
c    0.0
Name: pos_coverage, dtype: float64
a    0.4
b    1.0
c    0.6
Name: pos_coverage, dtype: float64
a    0.4
b    1.0
c    0.6
Name: pos_coverage, dtype: float64
a    0.4
b    1.0
c    0.6
Name: pos_coverage, dtype: float64
group
first     0.7
second    0.6
Name: pos_coverage, dtype: float64
group
first     0.7
second    0.6
Name: pos_coverage, dtype: float64
group
first     0.7
second    0.6
Name: pos_coverage, dtype: float64


In [83]:
%timeit big_portfolio.pos_coverage()
%timeit big_portfolio_grouped.pos_coverage(group_by=False)
%timeit big_portfolio_shared.pos_coverage(group_by=False)

%timeit big_portfolio.pos_coverage(group_by=big_group_by)
%timeit big_portfolio_grouped.pos_coverage()
%timeit big_portfolio_shared.pos_coverage()

16.6 ms ± 1.51 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
14.8 ms ± 104 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
15 ms ± 116 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
34.1 ms ± 1.49 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
32.2 ms ± 1.15 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
33.4 ms ± 1.19 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [84]:
print(portfolio.cash_flow(short_cash=False))
print(portfolio.cash_flow())
print(portfolio_grouped.cash_flow(group_by=False))
print(portfolio_shared.cash_flow(group_by=False))

print(portfolio.cash_flow(group_by=group_by))
print(portfolio_grouped.cash_flow())
print(portfolio_shared.cash_flow())

                  a        b        c
2020-01-01  0.00000 -1.09990 -1.12010
2020-01-02 -0.30402 -0.29998 -0.30402
2020-01-03  0.19403  0.00000  2.84030
2020-01-04  0.00000  0.29996  0.29204
2020-01-05 -5.20050 -5.09950  0.00000
                  a        b        c
2020-01-01  0.00000  0.88010 -1.12010
2020-01-02 -0.30402  0.09602 -0.30402
2020-01-03  0.19403  0.00000  2.84030
2020-01-04  0.00000 -0.50804  0.29204
2020-01-05 -5.20050  4.80050  0.00000
                  a        b        c
2020-01-01  0.00000  0.88010 -1.12010
2020-01-02 -0.30402  0.09602 -0.30402
2020-01-03  0.19403  0.00000  2.84030
2020-01-04  0.00000 -0.50804  0.29204
2020-01-05 -5.20050  4.80050  0.00000
                  a        b        c
2020-01-01  0.00000  0.88010 -1.12010
2020-01-02 -0.30402  0.09602 -0.30402
2020-01-03  0.19403  0.00000  2.84030
2020-01-04  0.00000 -0.50804  0.29204
2020-01-05 -5.20050  4.80050  0.00000
group         first   second
2020-01-01  0.88010 -1.12010
2020-01-02 -0.20800 -0.30402
2

In [85]:
%timeit big_portfolio.cash_flow()
%timeit big_portfolio.cash_flow(short_cash=False)
%timeit big_portfolio_grouped.cash_flow(group_by=False)
%timeit big_portfolio_shared.cash_flow(group_by=False)

%timeit big_portfolio.cash_flow(group_by=big_group_by)
%timeit big_portfolio_grouped.cash_flow()
%timeit big_portfolio_shared.cash_flow()

12.5 ms ± 502 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
14.6 ms ± 3.25 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
12.3 ms ± 175 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
11.8 ms ± 14.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
14.7 ms ± 56.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
14 ms ± 51.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
14.6 ms ± 41.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [86]:
print(portfolio.get_init_cash())
print(portfolio_grouped.get_init_cash(group_by=False))
print(portfolio_shared.get_init_cash(group_by=False))

print(portfolio.get_init_cash(group_by=group_by))
print(portfolio_grouped.get_init_cash())
print(portfolio_shared.get_init_cash())

a    100.0
b    100.0
c    100.0
Name: init_cash, dtype: float64
a    100.0
b    100.0
c    100.0
Name: init_cash, dtype: float64
a    200.0
b    200.0
c    100.0
Name: init_cash, dtype: float64
group
first     200.0
second    100.0
Name: init_cash, dtype: float64
group
first     200.0
second    100.0
Name: init_cash, dtype: float64
group
first     200.0
second    100.0
Name: init_cash, dtype: float64


In [87]:
%timeit big_portfolio.get_init_cash()
%timeit big_portfolio_grouped.get_init_cash(group_by=False)
%timeit big_portfolio_shared.get_init_cash(group_by=False)

%timeit big_portfolio.get_init_cash(group_by=big_group_by)
%timeit big_portfolio_grouped.get_init_cash()
%timeit big_portfolio_shared.get_init_cash()

43.2 µs ± 125 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
42.9 µs ± 611 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
533 µs ± 13.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
737 µs ± 44.3 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
46.6 µs ± 617 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
850 µs ± 37.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [7]:
print(vbt.Portfolio.from_orders(
    price_na, 1000., init_cash=InitCashMode.Auto, group_by=None).init_cash)

print(vbt.Portfolio.from_orders(
    price_na, 1000., init_cash=InitCashMode.Auto, group_by=group_by).init_cash)

print(vbt.Portfolio.from_orders(
    price_na, 1000., init_cash=InitCashMode.Auto, group_by=group_by, cash_sharing=True).init_cash)

print(vbt.Portfolio.from_orders(
    price_na, 1000., init_cash=InitCashMode.AutoAlign, group_by=None).init_cash)

print(vbt.Portfolio.from_orders(
    price_na, 1000., init_cash=InitCashMode.AutoAlign, group_by=group_by).init_cash)

print(vbt.Portfolio.from_orders(
    price_na, 1000., init_cash=InitCashMode.AutoAlign, group_by=group_by, cash_sharing=True).init_cash)

a    14000.0
b    12000.0
c    10000.0
Name: init_cash, dtype: float64
group
first     26000.0
second    10000.0
Name: init_cash, dtype: float64
group
first     26000.0
second    10000.0
Name: init_cash, dtype: float64
a    14000.0
b    14000.0
c    14000.0
Name: init_cash, dtype: float64
group
first     26000.0
second    26000.0
Name: init_cash, dtype: float64
group
first     26000.0
second    26000.0
Name: init_cash, dtype: float64


In [8]:
print(portfolio.cash(short_cash=False))
print(portfolio.cash())
print(portfolio_grouped.cash(group_by=False))
print(portfolio_shared.cash(group_by=False))
print(portfolio_shared.cash(group_by=False, in_sim_order=True))

print(portfolio.cash(group_by=group_by))
print(portfolio_grouped.cash())
print(portfolio_shared.cash())

                    a         b          c
2020-01-01  100.00000  98.90010   98.87990
2020-01-02   99.69598  98.60012   98.57588
2020-01-03   99.89001  98.60012  101.41618
2020-01-04   99.89001  98.90008  101.70822
2020-01-05   94.68951  93.80058  101.70822
                    a          b          c
2020-01-01  100.00000  100.88010   98.87990
2020-01-02   99.69598  100.97612   98.57588
2020-01-03   99.89001  100.97612  101.41618
2020-01-04   99.89001  100.46808  101.70822
2020-01-05   94.68951  105.26858  101.70822
                    a          b          c
2020-01-01  100.00000  100.88010   98.87990
2020-01-02   99.69598  100.97612   98.57588
2020-01-03   99.89001  100.97612  101.41618
2020-01-04   99.89001  100.46808  101.70822
2020-01-05   94.68951  105.26858  101.70822
                    a          b          c
2020-01-01  200.00000  200.88010   98.87990
2020-01-02  199.69598  200.97612   98.57588
2020-01-03  199.89001  200.97612  101.41618
2020-01-04  199.89001  200.46808  101.

In [90]:
%timeit big_portfolio.cash()
%timeit big_portfolio.cash(short_cash=False)
%timeit big_portfolio_grouped.cash(group_by=False)
%timeit big_portfolio_shared.cash(group_by=False)
%timeit big_portfolio_shared.cash(group_by=False, in_sim_order=True)

%timeit big_portfolio.cash(group_by=big_group_by)
%timeit big_portfolio_grouped.cash()
%timeit big_portfolio_shared.cash()

19.4 ms ± 920 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
18.3 ms ± 470 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
18.4 ms ± 2.12 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
17.6 ms ± 201 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
18.7 ms ± 303 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
18.3 ms ± 292 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
19.6 ms ± 1.75 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
21.7 ms ± 1.76 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [91]:
print(portfolio.holding_value(direction='longonly'))
print(portfolio.holding_value(direction='shortonly'))

print(portfolio.holding_value())
print(portfolio_grouped.holding_value(group_by=False))
print(portfolio_shared.holding_value(group_by=False))

print(portfolio.holding_value(group_by=group_by))
print(portfolio_grouped.holding_value())
print(portfolio_shared.holding_value())

              a    b    c
2020-01-01  0.0  0.0  1.0
2020-01-02  0.2  0.0  2.2
2020-01-03  0.0  0.0  0.3
2020-01-04  0.0  0.0  0.0
2020-01-05  5.0  0.0  0.0
              a     b    c
2020-01-01  0.0   1.0  0.0
2020-01-02  0.0   2.2  0.0
2020-01-03  0.0   NaN  0.0
2020-01-04  0.0   4.0  0.0
2020-01-05  0.0  10.0  0.0
              a     b    c
2020-01-01  0.0  -1.0  1.0
2020-01-02  0.2  -2.2  2.2
2020-01-03  0.0   NaN  0.3
2020-01-04  0.0  -4.0  0.0
2020-01-05  5.0 -10.0  0.0
              a     b    c
2020-01-01  0.0  -1.0  1.0
2020-01-02  0.2  -2.2  2.2
2020-01-03  0.0   NaN  0.3
2020-01-04  0.0  -4.0  0.0
2020-01-05  5.0 -10.0  0.0
              a     b    c
2020-01-01  0.0  -1.0  1.0
2020-01-02  0.2  -2.2  2.2
2020-01-03  0.0   NaN  0.3
2020-01-04  0.0  -4.0  0.0
2020-01-05  5.0 -10.0  0.0
group       first  second
2020-01-01   -1.0     1.0
2020-01-02   -2.0     2.2
2020-01-03    NaN     0.3
2020-01-04   -4.0     0.0
2020-01-05   -5.0     0.0
group       first  second
2020-01-01   -

In [92]:
%timeit big_portfolio.holding_value()
%timeit big_portfolio_grouped.holding_value(group_by=False)
%timeit big_portfolio_shared.holding_value(group_by=False)

%timeit big_portfolio.holding_value(group_by=big_group_by)
%timeit big_portfolio_grouped.holding_value()
%timeit big_portfolio_shared.holding_value()

16.7 ms ± 331 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
19.4 ms ± 2.51 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
18.4 ms ± 1.48 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)
34.4 ms ± 308 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
33.6 ms ± 428 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
34.7 ms ± 213 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
print(portfolio.gross_exposure(direction='longonly'))
print(portfolio.gross_exposure(direction='shortonly'))

print(portfolio.gross_exposure())
print(portfolio_grouped.gross_exposure(group_by=False))
print(portfolio_shared.gross_exposure(group_by=False))

print(portfolio.gross_exposure(group_by=group_by))
print(portfolio_grouped.gross_exposure())
print(portfolio_shared.gross_exposure())

                   a    b         c
2020-01-01  0.000000  0.0  0.010012
2020-01-02  0.002002  0.0  0.021831
2020-01-03  0.000000  0.0  0.002949
2020-01-04  0.000000  0.0  0.000000
2020-01-05  0.050156  0.0  0.000000
              a         b    c
2020-01-01  0.0  0.010010  0.0
2020-01-02  0.0  0.021825  0.0
2020-01-03  0.0       NaN  0.0
2020-01-04  0.0  0.038873  0.0
2020-01-05  0.0  0.096339  0.0
                   a         b         c
2020-01-01  0.000000 -0.010214  0.010012
2020-01-02  0.002002 -0.022822  0.021831
2020-01-03  0.000000       NaN  0.002949
2020-01-04  0.000000 -0.042150  0.000000
2020-01-05  0.050156 -0.119331  0.000000
                   a         b         c
2020-01-01  0.000000 -0.010214  0.010012
2020-01-02  0.002002 -0.022822  0.021831
2020-01-03  0.000000       NaN  0.002949
2020-01-04  0.000000 -0.042150  0.000000
2020-01-05  0.050156 -0.119331  0.000000
                   a         b         c
2020-01-01  0.000000 -0.005053  0.010012
2020-01-02  0.001001 -0.

In [10]:
%timeit big_portfolio.gross_exposure()
%timeit big_portfolio_grouped.gross_exposure(group_by=False)
%timeit big_portfolio_shared.gross_exposure(group_by=False)

%timeit big_portfolio.gross_exposure(group_by=big_group_by)
%timeit big_portfolio_grouped.gross_exposure()
%timeit big_portfolio_shared.gross_exposure()

54.6 ms ± 5.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
44.3 ms ± 1.64 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
44.2 ms ± 1.07 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
58.8 ms ± 2.64 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
55 ms ± 1.09 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
60.9 ms ± 1.87 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
print(portfolio.net_exposure())
print(portfolio_grouped.net_exposure(group_by=False))
print(portfolio_shared.net_exposure(group_by=False))

print(portfolio.net_exposure(group_by=group_by))
print(portfolio_grouped.net_exposure())
print(portfolio_shared.net_exposure())

                   a         b         c
2020-01-01  0.000000 -0.010010  0.010012
2020-01-02  0.002002 -0.021825  0.021831
2020-01-03  0.000000       NaN  0.002949
2020-01-04  0.000000 -0.038873  0.000000
2020-01-05  0.050156 -0.096339  0.000000
                   a         b         c
2020-01-01  0.000000 -0.010010  0.010012
2020-01-02  0.002002 -0.021825  0.021831
2020-01-03  0.000000       NaN  0.002949
2020-01-04  0.000000 -0.038873  0.000000
2020-01-05  0.050156 -0.096339  0.000000
                   a         b         c
2020-01-01  0.000000 -0.005002  0.010012
2020-01-02  0.001001 -0.010956  0.021831
2020-01-03  0.000000       NaN  0.002949
2020-01-04  0.000000 -0.019714  0.000000
2020-01-05  0.025039 -0.049068  0.000000
group          first    second
2020-01-01 -0.004953  0.010012
2020-01-02 -0.009849  0.021831
2020-01-03       NaN  0.002949
2020-01-04 -0.019573  0.000000
2020-01-05 -0.023233  0.000000
group          first    second
2020-01-01 -0.004953  0.010012
2020-01-02 -0.

In [12]:
%timeit big_portfolio.net_exposure()
%timeit big_portfolio_grouped.net_exposure(group_by=False)
%timeit big_portfolio_shared.net_exposure(group_by=False)

%timeit big_portfolio.net_exposure(group_by=big_group_by)
%timeit big_portfolio_grouped.net_exposure()
%timeit big_portfolio_shared.net_exposure()

91.5 ms ± 1.76 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
90.4 ms ± 1.52 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
95.4 ms ± 4.26 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
129 ms ± 3.21 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
119 ms ± 1.51 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
127 ms ± 1.85 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
print(portfolio.value())
print(portfolio_grouped.value(group_by=False))
print(portfolio_shared.value(group_by=False))
print(portfolio_shared.value(group_by=False, in_sim_order=True))

print(portfolio.value(group_by=group_by))
print(portfolio_grouped.value())
print(portfolio_shared.value())

                    a         b          c
2020-01-01  100.00000  99.88010   99.87990
2020-01-02   99.89598  98.77612  100.77588
2020-01-03   99.89001       NaN  101.71618
2020-01-04   99.89001  96.46808  101.70822
2020-01-05   99.68951  95.26858  101.70822
                    a         b          c
2020-01-01  100.00000  99.88010   99.87990
2020-01-02   99.89598  98.77612  100.77588
2020-01-03   99.89001       NaN  101.71618
2020-01-04   99.89001  96.46808  101.70822
2020-01-05   99.68951  95.26858  101.70822
                    a          b          c
2020-01-01  200.00000  199.88010   99.87990
2020-01-02  199.89598  198.77612  100.77588
2020-01-03  199.89001        NaN  101.71618
2020-01-04  199.89001  196.46808  101.70822
2020-01-05  199.68951  195.26858  101.70822
                    a          b          c
2020-01-01  199.88010  199.88010   99.87990
2020-01-02  198.67210  198.77612  100.77588
2020-01-03        NaN        NaN  101.71618
2020-01-04  196.35809  196.35809  101.70822


In [14]:
%timeit big_portfolio.value()
%timeit big_portfolio_grouped.value(group_by=False)
%timeit big_portfolio_shared.value(group_by=False)
%timeit big_portfolio_shared.value(group_by=False, in_sim_order=True)

%timeit big_portfolio.value(group_by=big_group_by)
%timeit big_portfolio_grouped.value()
%timeit big_portfolio_shared.value()

36.9 ms ± 1.21 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
35 ms ± 210 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
35.8 ms ± 81.1 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
46 ms ± 152 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
53.9 ms ± 626 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
50.4 ms ± 722 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
55.3 ms ± 941 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [15]:
print(portfolio.total_profit())
print(portfolio_grouped.total_profit(group_by=False))
print(portfolio_shared.total_profit(group_by=False))

print(portfolio.total_profit(group_by=group_by))
print(portfolio_grouped.total_profit())
print(portfolio_shared.total_profit())

a   -0.31049
b   -4.73142
c    1.70822
Name: total_profit, dtype: float64
a   -0.31049
b   -4.73142
c    1.70822
Name: total_profit, dtype: float64
a   -0.31049
b   -4.73142
c    1.70822
Name: total_profit, dtype: float64
group
first    -5.04191
second    1.70822
Name: total_profit, dtype: float64
group
first    -5.04191
second    1.70822
Name: total_profit, dtype: float64
group
first    -5.04191
second    1.70822
Name: total_profit, dtype: float64


In [16]:
%timeit big_portfolio.total_profit()
%timeit big_portfolio_grouped.total_profit(group_by=False)
%timeit big_portfolio_shared.total_profit(group_by=False)

%timeit big_portfolio.total_profit(group_by=big_group_by)
%timeit big_portfolio_grouped.total_profit()
%timeit big_portfolio_shared.total_profit()

12.3 ms ± 79.7 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
12.5 ms ± 190 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
12.5 ms ± 225 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
13.2 ms ± 224 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
12.6 ms ± 176 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
13.4 ms ± 217 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
print(portfolio.final_value())
print(portfolio_grouped.final_value(group_by=False))
print(portfolio_shared.final_value(group_by=False))

print(portfolio.final_value(group_by=group_by))
print(portfolio_grouped.final_value())
print(portfolio_shared.final_value())

a     99.68951
b     95.26858
c    101.70822
Name: final_value, dtype: float64
a     99.68951
b     95.26858
c    101.70822
Name: final_value, dtype: float64
a    199.68951
b    195.26858
c    101.70822
Name: final_value, dtype: float64
group
first     194.95809
second    101.70822
Name: final_value, dtype: float64
group
first     194.95809
second    101.70822
Name: final_value, dtype: float64
group
first     194.95809
second    101.70822
Name: final_value, dtype: float64


In [18]:
%timeit big_portfolio.final_value()
%timeit big_portfolio_grouped.final_value(group_by=False)
%timeit big_portfolio_shared.final_value(group_by=False)

%timeit big_portfolio.final_value(group_by=big_group_by)
%timeit big_portfolio_grouped.final_value()
%timeit big_portfolio_shared.final_value()

12.3 ms ± 159 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
12.2 ms ± 122 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
12.9 ms ± 128 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
14.1 ms ± 280 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
12 ms ± 145 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
14.1 ms ± 231 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [19]:
print(portfolio.total_return())
print(portfolio_grouped.total_return(group_by=False))
print(portfolio_shared.total_return(group_by=False))

print(portfolio.total_return(group_by=group_by))
print(portfolio_grouped.total_return())
print(portfolio_shared.total_return())

a   -0.003105
b   -0.047314
c    0.017082
Name: total_return, dtype: float64
a   -0.003105
b   -0.047314
c    0.017082
Name: total_return, dtype: float64
a   -0.001552
b   -0.023657
c    0.017082
Name: total_return, dtype: float64
group
first    -0.025210
second    0.017082
Name: total_return, dtype: float64
group
first    -0.025210
second    0.017082
Name: total_return, dtype: float64
group
first    -0.025210
second    0.017082
Name: total_return, dtype: float64


In [20]:
%timeit big_portfolio.total_return()
%timeit big_portfolio_grouped.total_return(group_by=False)
%timeit big_portfolio_shared.total_return(group_by=False)

%timeit big_portfolio.total_return(group_by=big_group_by)
%timeit big_portfolio_grouped.total_return()
%timeit big_portfolio_shared.total_return()

12.5 ms ± 200 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
12.4 ms ± 207 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
12.8 ms ± 151 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
14 ms ± 178 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
12.6 ms ± 199 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
14.2 ms ± 122 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [21]:
print(portfolio.returns())
print(portfolio_grouped.returns(group_by=False))
print(portfolio_shared.returns(group_by=False))
print(portfolio_shared.returns(group_by=False, in_sim_order=True))

print(portfolio.returns(group_by=group_by))
print(portfolio_grouped.returns())
print(portfolio_shared.returns())

                   a         b         c
2020-01-01  0.000000 -0.001199 -0.001201
2020-01-02 -0.001040 -0.011053  0.008971
2020-01-03 -0.000060       NaN  0.009331
2020-01-04  0.000000       NaN -0.000078
2020-01-05 -0.002007 -0.012434  0.000000
                   a         b         c
2020-01-01  0.000000 -0.001199 -0.001201
2020-01-02 -0.001040 -0.011053  0.008971
2020-01-03 -0.000060       NaN  0.009331
2020-01-04  0.000000       NaN -0.000078
2020-01-05 -0.002007 -0.012434  0.000000
                   a         b         c
2020-01-01  0.000000 -0.000600 -0.001201
2020-01-02 -0.000520 -0.005523  0.008971
2020-01-03 -0.000030       NaN  0.009331
2020-01-04  0.000000       NaN -0.000078
2020-01-05 -0.001003 -0.006105  0.000000
                   a         b         c
2020-01-01  0.000000 -0.000600 -0.001201
2020-01-02 -0.000523 -0.005523  0.008971
2020-01-03       NaN       NaN  0.009331
2020-01-04  0.000000       NaN -0.000078
2020-01-05 -0.001027 -0.006109  0.000000
group          f

In [22]:
%timeit big_portfolio.returns()
%timeit big_portfolio_grouped.returns(group_by=False)
%timeit big_portfolio_shared.returns(group_by=False)
%timeit big_portfolio_shared.returns(group_by=False, in_sim_order=True)

%timeit big_portfolio.returns(group_by=big_group_by)
%timeit big_portfolio_grouped.returns()
%timeit big_portfolio_shared.returns()

39.5 ms ± 2.59 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
37.4 ms ± 313 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
39.4 ms ± 226 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
52.3 ms ± 149 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
55.6 ms ± 310 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
50.9 ms ± 276 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
63.5 ms ± 4.23 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [23]:
print(portfolio.active_returns())
print(portfolio_grouped.active_returns(group_by=False))
print(portfolio_shared.active_returns(group_by=False))

print(portfolio.active_returns(group_by=group_by))
print(portfolio_grouped.active_returns())
print(portfolio_shared.active_returns())

                  a         b         c
2020-01-01  0.00000      -inf      -inf
2020-01-02     -inf -1.103980  0.895980
2020-01-03 -0.02985       NaN  0.427409
2020-01-04  0.00000       NaN -0.026533
2020-01-05     -inf -0.299875  0.000000
                  a         b         c
2020-01-01  0.00000      -inf      -inf
2020-01-02     -inf -1.103980  0.895980
2020-01-03 -0.02985       NaN  0.427409
2020-01-04  0.00000       NaN -0.026533
2020-01-05     -inf -0.299875  0.000000
                  a         b         c
2020-01-01  0.00000      -inf      -inf
2020-01-02     -inf -1.103980  0.895980
2020-01-03 -0.02985       NaN  0.427409
2020-01-04  0.00000       NaN -0.026533
2020-01-05     -inf -0.299875  0.000000
group       first    second
2020-01-01   -inf      -inf
2020-01-02 -1.208  0.895980
2020-01-03    NaN  0.427409
2020-01-04    NaN -0.026533
2020-01-05 -0.350  0.000000
group       first    second
2020-01-01   -inf      -inf
2020-01-02 -1.208  0.895980
2020-01-03    NaN  0.427409


In [24]:
%timeit big_portfolio.active_returns()
%timeit big_portfolio_grouped.active_returns(group_by=False)
%timeit big_portfolio_shared.active_returns(group_by=False)

%timeit big_portfolio.active_returns(group_by=big_group_by)
%timeit big_portfolio_grouped.active_returns()
%timeit big_portfolio_shared.active_returns()

40.2 ms ± 5.18 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
36.9 ms ± 235 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
37.4 ms ± 193 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
51.7 ms ± 195 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
49.8 ms ± 220 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
52.4 ms ± 264 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [25]:
print(portfolio.market_value())
print(portfolio_grouped.market_value(group_by=False))
print(portfolio_shared.market_value(group_by=False))

print(portfolio.market_value(group_by=group_by))
print(portfolio_grouped.market_value())
print(portfolio_shared.market_value())

                a      b      c
2020-01-01  100.0  100.0  100.0
2020-01-02  100.0  200.0  200.0
2020-01-03  150.0  200.0  300.0
2020-01-04  200.0  400.0  400.0
2020-01-05  250.0  500.0  400.0
                a      b      c
2020-01-01  100.0  100.0  100.0
2020-01-02  100.0  200.0  200.0
2020-01-03  150.0  200.0  300.0
2020-01-04  200.0  400.0  400.0
2020-01-05  250.0  500.0  400.0
                a       b      c
2020-01-01  200.0   200.0  100.0
2020-01-02  200.0   400.0  200.0
2020-01-03  300.0   400.0  300.0
2020-01-04  400.0   800.0  400.0
2020-01-05  500.0  1000.0  400.0
group       first  second
2020-01-01  200.0   100.0
2020-01-02  300.0   200.0
2020-01-03  350.0   300.0
2020-01-04  600.0   400.0
2020-01-05  750.0   400.0
group       first  second
2020-01-01  200.0   100.0
2020-01-02  300.0   200.0
2020-01-03  350.0   300.0
2020-01-04  600.0   400.0
2020-01-05  750.0   400.0
group       first  second
2020-01-01  200.0   100.0
2020-01-02  300.0   200.0
2020-01-03  350.0   300.0
20

In [26]:
%timeit big_portfolio.market_value()
%timeit big_portfolio_grouped.market_value(group_by=False)
%timeit big_portfolio_shared.market_value(group_by=False)

%timeit big_portfolio.market_value(group_by=big_group_by)
%timeit big_portfolio_grouped.market_value()
%timeit big_portfolio_shared.market_value()

909 µs ± 51.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
967 µs ± 34.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
1.81 ms ± 111 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
4.86 ms ± 415 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
2.82 ms ± 35.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
5.2 ms ± 387 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [27]:
print(portfolio.market_returns())
print(portfolio_grouped.market_returns(group_by=False))
print(portfolio_shared.market_returns(group_by=False))

print(portfolio.market_returns(group_by=group_by))
print(portfolio_grouped.market_returns())
print(portfolio_shared.market_returns())

                   a     b         c
2020-01-01  0.000000  0.00  0.000000
2020-01-02  0.000000  1.00  1.000000
2020-01-03  0.500000  0.00  0.500000
2020-01-04  0.333333  1.00  0.333333
2020-01-05  0.250000  0.25  0.000000
                   a     b         c
2020-01-01  0.000000  0.00  0.000000
2020-01-02  0.000000  1.00  1.000000
2020-01-03  0.500000  0.00  0.500000
2020-01-04  0.333333  1.00  0.333333
2020-01-05  0.250000  0.25  0.000000
                   a     b         c
2020-01-01  0.000000  0.00  0.000000
2020-01-02  0.000000  1.00  1.000000
2020-01-03  0.500000  0.00  0.500000
2020-01-04  0.333333  1.00  0.333333
2020-01-05  0.250000  0.25  0.000000
group          first    second
2020-01-01  0.000000  0.000000
2020-01-02  0.500000  1.000000
2020-01-03  0.166667  0.500000
2020-01-04  0.714286  0.333333
2020-01-05  0.250000  0.000000
group          first    second
2020-01-01  0.000000  0.000000
2020-01-02  0.500000  1.000000
2020-01-03  0.166667  0.500000
2020-01-04  0.714286  0.

In [28]:
%timeit big_portfolio.market_returns()
%timeit big_portfolio_grouped.market_returns(group_by=False)
%timeit big_portfolio_shared.market_returns(group_by=False)

%timeit big_portfolio.market_returns(group_by=big_group_by)
%timeit big_portfolio_grouped.market_returns()
%timeit big_portfolio_shared.market_returns()

3.7 ms ± 67.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
3.72 ms ± 66.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
5.11 ms ± 5.82 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
7.2 ms ± 618 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
4.1 ms ± 52.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
7.28 ms ± 40.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [29]:
print(portfolio.total_market_return())
print(portfolio_grouped.total_market_return(group_by=False))
print(portfolio_shared.total_market_return(group_by=False))

print(portfolio.total_market_return(group_by=group_by))
print(portfolio_grouped.total_market_return())
print(portfolio_shared.total_market_return())

a    1.5
b    4.0
c    3.0
Name: total_market_return, dtype: float64
a    1.5
b    4.0
c    3.0
Name: total_market_return, dtype: float64
a    1.5
b    4.0
c    3.0
Name: total_market_return, dtype: float64
group
first     2.75
second    3.00
Name: total_market_return, dtype: float64
group
first     2.75
second    3.00
Name: total_market_return, dtype: float64
group
first     2.75
second    3.00
Name: total_market_return, dtype: float64


In [30]:
%timeit big_portfolio.total_market_return()
%timeit big_portfolio_grouped.total_market_return(group_by=False)
%timeit big_portfolio_shared.total_market_return(group_by=False)

%timeit big_portfolio.total_market_return(group_by=big_group_by)
%timeit big_portfolio_grouped.total_market_return()
%timeit big_portfolio_shared.total_market_return()

975 µs ± 61.3 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
937 µs ± 10.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
1.83 ms ± 7.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
4.77 ms ± 30 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
2.85 ms ± 11.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
5.21 ms ± 79.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [31]:
print(portfolio_shared.cumulative_returns())
print(portfolio_shared.cumulative_returns(group_by=False))

group          first    second
2020-01-01 -0.000599 -0.001201
2020-01-02 -0.006639  0.007759
2020-01-03 -0.006639  0.017162
2020-01-04 -0.006639  0.017082
2020-01-05 -0.013722  0.017082
                   a         b         c
2020-01-01  0.000000 -0.000599 -0.001201
2020-01-02 -0.000520 -0.006119  0.007759
2020-01-03 -0.000550 -0.006119  0.017162
2020-01-04 -0.000550 -0.006119  0.017082
2020-01-05 -0.001552 -0.012187  0.017082


In [32]:
print(portfolio_shared.sharpe_ratio())
print(portfolio_shared.sharpe_ratio(risk_free=0.01))
print(portfolio_shared.sharpe_ratio(year_freq='252D'))
print(portfolio_shared.sharpe_ratio(group_by=False))

group
first    -25.066399
second    12.345065
Name: sharpe_ratio, dtype: float64
group
first    -79.665578
second   -23.917188
Name: sharpe_ratio, dtype: float64
group
first    -20.827915
second    10.257635
Name: sharpe_ratio, dtype: float64
a   -13.309506
b   -25.744690
c    12.345065
Name: sharpe_ratio, dtype: float64


In [33]:
print(portfolio.stats())  # mean statistics

Start                     2020-01-01 00:00:00
End                       2020-01-05 00:00:00
Duration                      5 days 00:00:00
Init. Cash                                100
Total Profit                         -1.11123
Total Return [%]                     -1.11123
Benchmark Return [%]                  283.333
Position Coverage [%]                 66.6667
Max. Drawdown [%]                     1.64512
Avg. Drawdown [%]                     1.64512
Max. Drawdown Duration        3 days 08:00:00
Avg. Drawdown Duration        3 days 08:00:00
Num. Trades                           1.33333
Win Rate [%]                          33.3333
Best Trade [%]                       -98.3806
Worst Trade [%]                      -100.804
Avg. Trade [%]                       -99.5922
Max. Trade Duration           2 days 08:00:00
Avg. Trade Duration           2 days 04:00:00
Expectancy                           0.108273
SQN                                   1.23509
Gross Exposure                    

/Users/olegpolakow/Documents/SourceTree/vectorbt/vectorbt/utils/decorators.py:306: UserWarning: Taking mean across columns. To return a DataFrame, pass agg_func=None.
  return func(instance, *args, **kwargs)


In [34]:
print(portfolio['a'].stats())

Start                     2020-01-01 00:00:00
End                       2020-01-05 00:00:00
Duration                      5 days 00:00:00
Init. Cash                                100
Total Profit                         -0.31049
Total Return [%]                     -0.31049
Benchmark Return [%]                      150
Position Coverage [%]                      40
Max. Drawdown [%]                     0.31049
Avg. Drawdown [%]                     0.31049
Max. Drawdown Duration        4 days 00:00:00
Avg. Drawdown Duration        4 days 00:00:00
Num. Trades                                 1
Win Rate [%]                                0
Best Trade [%]                       -54.4505
Worst Trade [%]                      -54.4505
Avg. Trade [%]                       -54.4505
Max. Trade Duration           1 days 00:00:00
Avg. Trade Duration           1 days 00:00:00
Expectancy                           -0.10999
SQN                                       NaN
Gross Exposure                    

In [35]:
print(portfolio['a'].stats(required_return=0.1, risk_free=0.01))  # test kwargs

Start                     2020-01-01 00:00:00
End                       2020-01-05 00:00:00
Duration                      5 days 00:00:00
Init. Cash                                100
Total Profit                         -0.31049
Total Return [%]                     -0.31049
Benchmark Return [%]                      150
Position Coverage [%]                      40
Max. Drawdown [%]                     0.31049
Avg. Drawdown [%]                     0.31049
Max. Drawdown Duration        4 days 00:00:00
Avg. Drawdown Duration        4 days 00:00:00
Num. Trades                                 1
Win Rate [%]                                0
Best Trade [%]                       -54.4505
Worst Trade [%]                      -54.4505
Avg. Trade [%]                       -54.4505
Max. Trade Duration           1 days 00:00:00
Avg. Trade Duration           1 days 00:00:00
Expectancy                           -0.10999
SQN                                       NaN
Gross Exposure                    

In [36]:
print(portfolio['a'].stats(incl_unrealized=True))  # test incl_unrealized

Start                     2020-01-01 00:00:00
End                       2020-01-05 00:00:00
Duration                      5 days 00:00:00
Init. Cash                                100
Total Profit                         -0.31049
Total Return [%]                     -0.31049
Benchmark Return [%]                      150
Position Coverage [%]                      40
Max. Drawdown [%]                     0.31049
Avg. Drawdown [%]                     0.31049
Max. Drawdown Duration        4 days 00:00:00
Avg. Drawdown Duration        4 days 00:00:00
Num. Trades                                 2
Win Rate [%]                                0
Best Trade [%]                        -3.9703
Worst Trade [%]                      -54.4505
Avg. Trade [%]                       -29.2104
Max. Trade Duration           1 days 00:00:00
Avg. Trade Duration           0 days 12:00:00
Expectancy                          -0.155245
SQN                                  -3.43045
Gross Exposure                    

In [37]:
portfolio_grouped['first'].stats()

Start                     2020-01-01 00:00:00
End                       2020-01-05 00:00:00
Duration                      5 days 00:00:00
Init. Cash                                200
Total Profit                         -5.04191
Total Return [%]                     -2.52095
Benchmark Return [%]                      275
Position Coverage [%]                      70
Max. Drawdown [%]                     2.46248
Avg. Drawdown [%]                     2.46248
Max. Drawdown Duration        4 days 00:00:00
Avg. Drawdown Duration        4 days 00:00:00
Num. Trades                                 2
Win Rate [%]                                0
Best Trade [%]                       -54.4505
Worst Trade [%]                      -388.242
Avg. Trade [%]                       -221.346
Max. Trade Duration           3 days 00:00:00
Avg. Trade Duration           2 days 00:00:00
Expectancy                          -0.264646
SQN                                  -1.71119
Gross Exposure                    

In [38]:
pd.testing.assert_series_equal(portfolio['c'].stats(), portfolio.stats(column='c'))
pd.testing.assert_series_equal(portfolio['c'].stats(), portfolio_grouped.stats(column='c', group_by=False))
pd.testing.assert_series_equal(portfolio_grouped['second'].stats(), portfolio_grouped.stats(column='second'))

In [39]:
# Calculates stats for one column
%timeit big_portfolio.iloc[0].stats()
%timeit big_portfolio_grouped.iloc[0].stats(group_by=False, column=0)
%timeit big_portfolio_shared.iloc[0].stats(group_by=False, column=0)

%timeit big_portfolio.iloc[0].stats(group_by=np.array([0]))
%timeit big_portfolio_grouped.iloc[0].stats()
%timeit big_portfolio_shared.iloc[0].stats()

19.7 ms ± 669 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
18.7 ms ± 425 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
31.2 ms ± 890 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
37.5 ms ± 276 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
19.2 ms ± 210 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
42.6 ms ± 956 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [40]:
# Calculates stats for all columns and selects one, takes advantage of caching (which is disabled here)
# Thus some series are re-calculated multiple times, such as records and returns
%timeit big_portfolio.stats(column=0)
%timeit big_portfolio_grouped.stats(group_by=False, column=0)
%timeit big_portfolio_shared.stats(group_by=False, column=0)

%timeit big_portfolio.stats(group_by=big_group_by, column=0)
%timeit big_portfolio_grouped.stats(column=0)
%timeit big_portfolio_shared.stats(column=0)

291 ms ± 3.22 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
305 ms ± 7.86 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
300 ms ± 5.05 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
361 ms ± 5.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
339 ms ± 3.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
361 ms ± 2.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [41]:
print(portfolio.returns_stats())  # mean statistics

Start                    2020-01-01 00:00:00
End                      2020-01-05 00:00:00
Duration                     5 days 00:00:00
Total Return [%]                    -0.35145
Benchmark Return [%]                 283.333
Annual Return [%]                    46.7874
Annual Volatility [%]                7.82518
Sharpe Ratio                        -8.87388
Calmar Ratio                           10374
Max. Drawdown [%]                  -0.884431
Omega Ratio                           4.7687
Sortino Ratio                        30.9251
Skew                                0.329244
Kurtosis                            -1.56615
Tail Ratio                           3.21985
Common Sense Ratio                   10.8931
Value at Risk                    -0.00502877
Alpha                              -0.424832
Beta                              0.00143011
Name: stats_mean, dtype: object


/Users/olegpolakow/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Taking mean across columns. To return a DataFrame, pass agg_func=None.
  """Entry point for launching an IPython kernel.


In [42]:
print(portfolio['a'].returns_stats())

Start                    2020-01-01 00:00:00
End                      2020-01-05 00:00:00
Duration                     5 days 00:00:00
Total Return [%]                    -0.31049
Benchmark Return [%]                     150
Annual Return [%]                   -20.3087
Annual Volatility [%]                1.70441
Sharpe Ratio                         -13.308
Calmar Ratio                        -65.4087
Max. Drawdown [%]                   -0.31049
Omega Ratio                                0
Sortino Ratio                       -11.7389
Skew                                -1.21911
Kurtosis                            0.122976
Tail Ratio                                 0
Common Sense Ratio                         0
Value at Risk                    -0.00181381
Alpha                              -0.248883
Beta                             0.000749314
Name: a, dtype: object


In [43]:
print(portfolio_grouped['first'].returns_stats())

Start                    2020-01-01 00:00:00
End                      2020-01-05 00:00:00
Duration                     5 days 00:00:00
Total Return [%]                     -1.3722
Benchmark Return [%]                     275
Annual Return [%]                   -63.5286
Annual Volatility [%]                6.68508
Sharpe Ratio                        -25.0664
Calmar Ratio                         -48.383
Max. Drawdown [%]                   -1.31304
Omega Ratio                                0
Sortino Ratio                       -16.2205
Skew                                 1.54618
Kurtosis                                 NaN
Tail Ratio                          0.162922
Common Sense Ratio                 0.0594201
Value at Risk                    -0.00702121
Alpha                              -0.494797
Beta                              -0.0108882
Name: first, dtype: object


In [44]:
pd.testing.assert_series_equal(portfolio['c'].returns_stats(), portfolio.returns_stats(column='c'))
pd.testing.assert_series_equal(portfolio['c'].returns_stats(), portfolio_grouped.returns_stats(column='c', group_by=False))
pd.testing.assert_series_equal(portfolio_grouped['second'].returns_stats(), portfolio_grouped.returns_stats(column='second'))

In [12]:
portfolio['a'].plot(subplots='all').show_svg()

In [13]:
portfolio_grouped['first'].plot(subplots='all').show_svg()

In [14]:
from vectorbt.utils.colors import adjust_opacity

portfolio['a'].plot(
    subplots=['drawdowns', 'underwater'],
    drawdowns_kwargs=dict(top_n=3),
    underwater_kwargs=dict(
        trace_kwargs=dict(
            line_color='#FF6F00',
            fillcolor=adjust_opacity('#FF6F00', 0.3)
        )
    )
).show_svg()

In [16]:
from vectorbt import settings
color_schema = settings['plotting']['color_schema']

fig = portfolio['a'].plot(subplots=[
    'orders',
    ('order_size', dict(title='Order Size', can_plot_groups=False))  # placeholder
])

order_size.vbt.plot(trace_kwargs=dict(name='Order Size'), add_trace_kwargs=dict(row=2, col=1), fig=fig).show_svg()